#**Project Structure and Folder Hierarchy**

This CRISP-DM pipeline follows a systematic folder organization within the ***/workspace/*** directory, mirroring the six core phases of the Cross-Industry Standard Process for Data Mining methodology. The project structure includes dedicated directories for each CRISP-DM phase: ***Data_Understanding/*** for exploratory data analysis and initial dataset investigation, ***Data_Preparation/*** for implementing the "single source of truth" strategy and cleaned dataset storage, ***Modeling/*** for comparative resampling experiments (SMOTE vs. Borderline-SMOTE) and algorithm evaluation, ***Evaluation/*** for cross-validation results and performance metrics analysis, and ***Deployment/*** for the final ensemble model artifact *(final_best_of_best_ensemble.pkl)*. <br>
Try to put the csv file of the dataset directly under workspace folder (you will find the file in the github). <br>
This hierarchical organization ensures systematic workflow management, maintains clear separation of concerns between different pipeline phases, and facilitates reproducible research practices essential for rigorous cybersecurity machine learning applications. The ***.ipynb_checkpoints/*** folder contains Jupyter notebook version control files, supporting iterative development and experimentation tracking throughout the project lifecycle.

# BUSINESS UNDERSTANDING


## 1.1 Project Objective

The strategic objective is to develop a high-performance machine learning model (which is an ensemble model in our case) capable of identifying and classifying network-based cyber-attacks against Unmanned Aerial Vehicles (UAVs) in real-time. The model serves as the intelligent core of an Intrusion Detection System (IDS) for UAV-Ground Control Station (GCS) communications, addressing critical cybersecurity vulnerabilities in UAV operations.

## 1.2 Business Problem

UAVs are increasingly integrated into critical infrastructure and commercial applications, making their communication links high-value targets for malicious actors. Current security solutions lack UAV-specific detection capabilities, leaving operations vulnerable to:

- **Data theft and operational disruption**
- **Loss of UAV control and mission compromise**  
- **GPS jamming/spoofing attacks**
- **Protocol-specific vulnerabilities (MAVLink, DJI SDK)**
- **Financial, reputational, and public safety risks**

## 1.3 Success Criteria

**Primary KPI**: Macro-averaged F1-score **>0.90**
- Chosen over accuracy due to severe class imbalance in cybersecurity data
- Ensures equal weight to rare but critical attack classes
- Aligns with business need to detect all threats, not just frequent ones

**Secondary Criteria**:
- **Model Robustness**: Stable performance across cross-validation folds
- **Interpretability**: Clear analysis of per-class performance and limitations

## 1.4 Dataset Foundation

**UAV Network Intrusion Detection Dataset (UAV-NIDD) - Scenario 1**
- **860,643 records, 45 features**
- **12 attack types + Normal traffic** (after excluding statistically unstable Reconnaissance class, n=6)
- **Real-world UAV network traffic** with UAV-specific protocols
- **Severe class imbalance**: 32,195:1 ratio between most/least frequent classes

## 1.5 Key Challenges

1. **Class Imbalance**: Requires sophisticated resampling techniques
2. **Protocol Specificity**: UAV-specific communication patterns
3. **Real-time Requirements**: Operational deployment needs
4. **Statistical Stability**: Scientific exclusion of unreliable minority classes

## 1.6 Expected Business Impact

- **Risk Mitigation**: Reduced successful cyber-attack rates
- **Operational Continuity**: Prevention of mission disruption
- **Regulatory Compliance**: Enhanced security for regulated environments
- **Technology Leadership**: Advanced UAV cybersecurity capabilities

This project focuses specifically on UAV-GCS communication security using CRISP-DM methodology to ensure systematic, scientifically valid development of a deployable intrusion detection system.

# DATA UNDERSTANDING

**Convert the Dataset from CSV to XLSX**

In [ ]:
import pandas as pd

def csv_to_excel(csv_path, excel_path, chunksize=100000):
    """
    Convert a large CSV file to Excel format in chunks.

    Args:
        csv_path (str): Path to the input CSV file.
        excel_path (str): Path to the output Excel file.
        chunksize (int): Number of rows per chunk to load.
    """
    writer = pd.ExcelWriter(excel_path, engine='openpyxl')  # engine='xlsxwriter' also works

    first_chunk = True
    chunk_num = 0

    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        print(f"Processing chunk {chunk_num + 1}...")

        if first_chunk:
            chunk.to_excel(writer, index=False, sheet_name='Sheet1', startrow=0)
            first_chunk = False
            startrow = chunk.shape[0] + 1
        else:
            chunk.to_excel(writer, index=False, header=False, sheet_name='Sheet1', startrow=startrow)
            startrow += chunk.shape[0]

        chunk_num += 1

    writer.close()
    print(f"\nConversion completed: {excel_path}")

# Exemple d'utilisation :
if __name__ == "__main__":
    csv_to_excel("/workspace/UAV-Case1-Label.csv", "/workspace/Dataset-NIDD.xlsx")

**UAV-NIDD Dashboard to Understand Our Data**

In [ ]:
# ====================================================================
# UAV-NIDD PROFESSIONAL DASHBOARD - COMPLETE FINAL VERSION
# ====================================================================

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import os
from datetime import datetime
import time
import warnings

print("Loading UAV-NIDD dataset for comprehensive analysis...")
try:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    from sklearn.feature_selection import SelectKBest, f_classif
    from scipy.stats import pearsonr
    print("Scikit-learn and scipy imported successfully")
except ImportError:
    print("Warning: Scikit-learn not available. Feature importance analysis will be skipped.")

warnings.filterwarnings('ignore')

# Dataset path - CHANGE THIS TO YOUR DATASET PATH
DATASET_PATH = "/workspace/Dataset-NIDD-with-category.xlsx"

print("Initializing comprehensive dataset analyzer...")

# ====================================================================
# ADVANCED FEATURE CORRELATION AND IMPORTANCE ANALYZER
# ====================================================================

class FeatureCorrelationImportanceAnalyzer:
    """Advanced analyzer for feature correlation and importance analysis"""

    def __init__(self, data, label_column, threshold=0.85):
        self.data = data.copy()
        self.label_column = label_column
        self.threshold = threshold
        self.results = {}
        self.scaler = StandardScaler()

        print(f"Feature Analyzer Initialized")
        print(f"   Dataset: {self.data.shape}")
        print(f"   Label: {self.label_column}")
        print(f"   Threshold: {self.threshold}")

    def prepare_data(self):
        """Prepare data for analysis"""
        print("\nPreparing data...")

        if self.label_column not in self.data.columns:
            raise ValueError(f"Label column '{self.label_column}' not found")

        # Get numeric features only
        numeric_features = self.data.select_dtypes(include=[np.number]).columns.tolist()
        if self.label_column in numeric_features:
            numeric_features.remove(self.label_column)

        self.X = self.data[numeric_features].copy()
        self.y = self.data[self.label_column].copy()

        print(f"   Numeric features: {len(numeric_features)}")

        # Handle missing values
        missing_before = self.X.isnull().sum().sum()
        if missing_before > 0:
            print(f"   Handling {missing_before} missing values...")
            self.X = self.X.fillna(self.X.mean())

        # Handle infinite values
        inf_count = np.isinf(self.X.values).sum()
        if inf_count > 0:
            print(f"   Handling {inf_count} infinite values...")
            self.X = self.X.replace([np.inf, -np.inf], np.nan)
            self.X = self.X.fillna(self.X.mean())

        # Encode target variable
        if self.y.dtype == 'object':
            self.label_encoder = LabelEncoder()
            self.y_encoded = self.label_encoder.fit_transform(self.y.astype(str))
            print(f"   Encoded {len(self.label_encoder.classes_)} target classes")
        else:
            self.y_encoded = self.y.values
            self.label_encoder = None

        print(f"Data preparation completed")
        return True

    def calculate_pearson_correlation(self):
        """Calculate Pearson correlation matrix"""
        print("\n🔗 Calculating Pearson Correlation...")
        start_time = time.time()

        # Sample for performance
        sample_size = min(50000, len(self.X))
        if sample_size < len(self.X):
            sample_indices = np.random.choice(len(self.X), sample_size, replace=False)
            X_sample = self.X.iloc[sample_indices]
            print(f"   Using sample of {sample_size:,} rows")
        else:
            X_sample = self.X

        # Calculate correlation matrix
        correlation_matrix = X_sample.corr(method='pearson')

        # Find highly correlated pairs
        high_correlation_pairs = []
        for i in range(len(correlation_matrix.columns)):
            for j in range(i+1, len(correlation_matrix.columns)):
                corr_value = correlation_matrix.iloc[i, j]
                if abs(corr_value) >= self.threshold:
                    high_correlation_pairs.append({
                        'feature_1': correlation_matrix.columns[i],
                        'feature_2': correlation_matrix.columns[j],
                        'correlation': corr_value,
                        'abs_correlation': abs(corr_value)
                    })

        # Sort by correlation
        high_correlation_pairs = sorted(high_correlation_pairs,
                                      key=lambda x: x['abs_correlation'], reverse=True)

        # Statistics
        correlation_stats = {
            'mean_correlation': correlation_matrix.abs().mean().mean(),
            'max_correlation': correlation_matrix.abs().max().max(),
            'highly_correlated_pairs': len(high_correlation_pairs),
            'redundant_features': len([pair for pair in high_correlation_pairs
                                     if pair['abs_correlation'] > 0.95])
        }

        processing_time = time.time() - start_time

        print(f"Correlation completed in {processing_time:.2f}s")
        print(f"Mean correlation: {correlation_stats['mean_correlation']:.3f}")
        print(f"High pairs (>{self.threshold}): {len(high_correlation_pairs)}")
        print(f"Redundant (>0.95): {correlation_stats['redundant_features']}")

        # Store results
        self.results['correlation'] = {
            'matrix': correlation_matrix,
            'high_pairs': high_correlation_pairs,
            'statistics': correlation_stats
        }

        return correlation_matrix, high_correlation_pairs

    def calculate_feature_importance(self):
        """Calculate feature importance using Random Forest"""
        print("\nCalculating Feature Importance...")
        start_time = time.time()

        # Sample for performance
        sample_size = min(50000, len(self.X))
        if sample_size < len(self.X):
            sample_indices = np.random.choice(len(self.X), sample_size, replace=False)
            X_sample = self.X.iloc[sample_indices]
            y_sample = self.y_encoded[sample_indices]
            print(f"   Using sample of {sample_size:,} rows")
        else:
            X_sample = self.X
            y_sample = self.y_encoded

        try:
            # Train Random Forest
            rf_classifier = RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                n_jobs=-1
            )

            rf_classifier.fit(X_sample, y_sample)

            # Get importances
            importances = rf_classifier.feature_importances_

            # Create dataframe
            feature_importance_df = pd.DataFrame({
                'feature': X_sample.columns,
                'importance': importances
            }).sort_values('importance', ascending=False)

            # Categorize by importance
            feature_importance_df['importance_level'] = feature_importance_df['importance'].apply(
                lambda x: 'Critical' if x > 0.05 else 'Moderate' if x > 0.02 else 'Low'
            )

            processing_time = time.time() - start_time

            print(f"Importance calculated in {processing_time:.2f}s")
            print(f" Top 5 features:")
            for i, row in feature_importance_df.head(5).iterrows():
                print(f"      {i+1}. {row['feature']}: {row['importance']:.4f} ({row['importance_level']})")

            # Store results
            self.results['importance'] = {
                'dataframe': feature_importance_df,
                'model': rf_classifier
            }

            return feature_importance_df

        except Exception as e:
            print(f" Error: {str(e)}")
            return None

    def calculate_anova_f_scores(self, k_best=25):
        """Calculate ANOVA F-scores"""
        print(f"\n Calculating ANOVA F-scores (top {k_best})...")
        start_time = time.time()

        try:
            # Apply SelectKBest
            selector = SelectKBest(score_func=f_classif, k=min(k_best, len(self.X.columns)))
            X_selected = selector.fit_transform(self.X, self.y_encoded)

            # Get scores
            feature_scores = selector.scores_
            selected_features = self.X.columns[selector.get_support()]

            # Create dataframe
            anova_scores_df = pd.DataFrame({
                'feature': self.X.columns,
                'f_score': feature_scores,
                'selected': selector.get_support()
            }).sort_values('f_score', ascending=False)

            processing_time = time.time() - start_time

            print(f"   ANOVA completed in {processing_time:.2f}s")
            print(f"   Top 5 by F-score:")
            for i, row in anova_scores_df.head(5).iterrows():
                status = " Selected" if row['selected'] else " Not selected"
                print(f"      {i+1}. {row['feature']}: {row['f_score']:.2e} ({status})")

            # Store results
            self.results['anova'] = {
                'dataframe': anova_scores_df,
                'selected_features': selected_features,
                'selector': selector
            }

            return anova_scores_df, selected_features

        except Exception as e:
            print(f" Error: {str(e)}")
            return None, None

    def identify_redundant_features(self):
        """Identify redundant features"""
        print("\n Identifying Redundant Features...")

        if 'correlation' not in self.results:
            print("Please run correlation analysis first")
            return []

        high_pairs = self.results['correlation']['high_pairs']
        features_to_remove = []

        # For each highly correlated pair
        for pair in high_pairs:
            if pair['abs_correlation'] > 0.95:
                feature1 = pair['feature_1']
                feature2 = pair['feature_2']

                # Keep more important feature
                if 'importance' in self.results:
                    importance_df = self.results['importance']['dataframe']
                    imp1 = importance_df[importance_df['feature'] == feature1]['importance'].iloc[0]
                    imp2 = importance_df[importance_df['feature'] == feature2]['importance'].iloc[0]

                    if imp1 > imp2:
                        if feature2 not in features_to_remove:
                            features_to_remove.append(feature2)
                    else:
                        if feature1 not in features_to_remove:
                            features_to_remove.append(feature1)
                else:
                    # Remove second feature
                    if feature2 not in features_to_remove:
                        features_to_remove.append(feature2)

        print(f" Identified {len(features_to_remove)} redundant features")
        if features_to_remove:
            print(f"   Features: {features_to_remove[:10]}{'...' if len(features_to_remove) > 10 else ''}")

        self.results['redundant_features'] = features_to_remove
        return features_to_remove

    def generate_feature_recommendations(self):
        """Generate recommendations"""
        print("\n Generating Recommendations...")

        recommendations = {
            'total_features': len(self.X.columns),
            'actions': {}
        }

        # Critical features
        if 'importance' in self.results:
            importance_df = self.results['importance']['dataframe']
            critical_features = importance_df[importance_df['importance'] > 0.05]['feature'].tolist()
            moderate_features = importance_df[
                (importance_df['importance'] > 0.02) &
                (importance_df['importance'] <= 0.05)
            ]['feature'].tolist()

            recommendations['actions']['keep_critical'] = critical_features
            recommendations['actions']['keep_moderate'] = moderate_features

            print(f" Critical: {len(critical_features)}")
            print(f" Moderate: {len(moderate_features)}")

        # Redundant features
        if 'redundant_features' in self.results:
            redundant = self.results['redundant_features']
            recommendations['actions']['remove_redundant'] = redundant
            print(f"  Redundant: {len(redundant)}")

        # Top ANOVA features
        if 'anova' in self.results:
            top_anova = self.results['anova']['selected_features'].tolist()
            recommendations['actions']['top_anova'] = top_anova
            print(f"    Top ANOVA: {len(top_anova)}")

        # Final recommendation
        if 'importance' in self.results and 'redundant_features' in self.results:
            keep_features = set(critical_features + moderate_features) - set(redundant)
            recommendations['final_recommended_features'] = list(keep_features)

            reduction_percentage = (1 - len(keep_features) / len(self.X.columns)) * 100
            print(f"    Final: Keep {len(keep_features)} features")
            print(f"    Reduction: {reduction_percentage:.1f}%")

        self.results['recommendations'] = recommendations
        return recommendations

    def run_complete_analysis(self, k_best=25):
        """Run complete analysis pipeline"""
        print("="*70)
        print(" STARTING FEATURE ANALYSIS")
        print("="*70)

        total_start_time = time.time()

        # Run analysis steps
        if not self.prepare_data():
            return False

        self.calculate_pearson_correlation()
        self.calculate_feature_importance()
        self.calculate_anova_f_scores(k_best)
        self.identify_redundant_features()
        self.generate_feature_recommendations()

        total_time = time.time() - total_start_time

        print(f"\n" + "="*70)
        print(" FEATURE ANALYSIS COMPLETED")
        print("="*70)
        print(f" Total time: {total_time:.2f} seconds")

        return True

# ====================================================================
# COMPREHENSIVE DATASET ANALYZER
# ====================================================================

class ComprehensiveDatasetAnalyzer:
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        self.data = None
        self.analysis_results = {}
        self.feature_analyzer = None

    def load_complete_dataset(self):
        """Load the complete dataset"""
        try:
            if not os.path.exists(self.dataset_path):
                print(f"Error: Dataset not found at {self.dataset_path}")
                return False

            print(f"Loading dataset: {os.path.basename(self.dataset_path)}")
            start_time = time.time()

            # Load dataset
            if self.dataset_path.endswith('.csv'):
                self.data = pd.read_csv(self.dataset_path, low_memory=False)
            else:
                self.data = pd.read_excel(self.dataset_path)

            load_time = time.time() - start_time
            file_size = os.path.getsize(self.dataset_path) / (1024*1024)

            print(f" Loaded in {load_time:.2f}s")
            print(f" Dataset: {len(self.data):,} samples x {len(self.data.columns)} features")
            print(f" File size: {file_size:.1f} MB")

            return True

        except Exception as e:
            print(f" Error loading: {str(e)}")
            return False

    def perform_comprehensive_analysis(self):
        """Perform complete analysis"""
        if self.data is None:
            print("No data loaded")
            return False

        print("Performing comprehensive analysis...")

        # Get columns
        columns = self.data.columns.tolist()
        print(f"Columns ({len(columns)}): {columns[:10]}{'...' if len(columns) > 10 else ''}")

        # Detect label and category columns
        label_col = None
        category_col = None

        # Exact match
        for col in columns:
            if col.lower() in ['label', 'attack', 'class']:
                label_col = col
            elif col.lower() in ['category', 'cat', 'type']:
                category_col = col

        # Partial match
        if not label_col:
            for col in columns:
                if 'label' in col.lower() or 'attack' in col.lower():
                    label_col = col
                    break

        if not category_col:
            for col in columns:
                if 'category' in col.lower() or 'cat' in col.lower():
                    category_col = col
                    break

        print(f"Label column: {label_col}")
        print(f"Category column: {category_col}")

        # Analyze distributions
        label_distribution = None
        category_distribution = None

        if label_col and label_col in self.data.columns:
            label_distribution = self.data[label_col].value_counts()
            print(f"\nAttack Types ({label_col}):")
            for i, (label, count) in enumerate(label_distribution.items()):
                percentage = count / len(self.data) * 100
                print(f"   {i+1}. {label}: {count:,} ({percentage:.1f}%)")

        if category_col and category_col in self.data.columns:
            category_distribution = self.data[category_col].value_counts()
            print(f"\nCategories ({category_col}):")
            for i, (cat, count) in enumerate(category_distribution.items()):
                percentage = count / len(self.data) * 100
                normal_indicator = " (Normal)" if 'normal' in str(cat).lower() else ""
                print(f"   {i+1}. {cat}: {count:,} ({percentage:.1f}%){normal_indicator}")

        # Calculate statistics
        total_samples = len(self.data)
        total_features = len(self.data.columns)
        numeric_cols = self.data.select_dtypes(include=[np.number]).columns.tolist()
        categorical_cols = self.data.select_dtypes(include=['object']).columns.tolist()

        # Imbalance ratio
        imbalance_ratio = 1
        if label_distribution is not None and len(label_distribution) > 1:
            imbalance_ratio = label_distribution.max() / label_distribution.min()

        # Data quality
        missing_data = self.data.isnull().sum()
        missing_percent = (missing_data / len(self.data) * 100).round(2)
        excellent_features = len(missing_percent[missing_percent == 0])
        poor_features = len(missing_percent[missing_percent > 50])

        # Store results
        self.analysis_results = {
            'basic': {
                'total_samples': total_samples,
                'total_features': total_features,
                'numeric_features': len(numeric_cols),
                'categorical_features': len(categorical_cols),
                'label_column': label_col,
                'category_column': category_col,
                'file_name': os.path.basename(self.dataset_path)
            },
            'labels': {
                'distribution': label_distribution,
                'num_types': len(label_distribution) if label_distribution is not None else 0,
                'imbalance_ratio': imbalance_ratio
            },
            'categories': {
                'distribution': category_distribution,
                'num_categories': len(category_distribution) if category_distribution is not None else 0
            },
            'quality': {
                'excellent_features': excellent_features,
                'poor_features': poor_features,
                'missing_percent': missing_percent
            }
        }

        print(f"\n Analysis completed:")
        print(f" Samples: {total_samples:,}")
        print(f" Attack types: {self.analysis_results['labels']['num_types']}")
        print(f" Categories: {self.analysis_results['categories']['num_categories']}")
        print(f"  Imbalance: {imbalance_ratio:.1f}:1")

        # Run advanced analyses
        print("\nRunning advanced analyses...")
        self.analysis_results['temporal'] = self.perform_temporal_analysis()
        self.analysis_results['wireless'] = self.perform_wireless_analysis()

        # Feature analysis
        if label_col:
            print("\nRunning Advanced Feature Analysis...")
            self.feature_analyzer = FeatureCorrelationImportanceAnalyzer(
                data=self.data,
                label_column=label_col,
                threshold=0.85
            )

            if self.feature_analyzer.run_complete_analysis(k_best=25):
                self.analysis_results['advanced_features'] = self.feature_analyzer.results
                print(" Advanced feature analysis completed")
            else:
                print(" Advanced feature analysis failed")
                self.analysis_results['advanced_features'] = {'error': 'Analysis failed'}
        else:
            print("  Skipping feature analysis - no label column")
            self.analysis_results['advanced_features'] = {'error': 'No label column'}

        return True


    def perform_wireless_analysis(self):
        """Advanced wireless characteristics analysis"""
        print("Performing wireless characteristics analysis...")

        wireless_results = {}

        # Find wireless-related columns
        signal_cols = [col for col in self.data.columns if any(kw in col.lower()
                      for kw in ['signal', 'dbm', 'rssi', 'power'])]

        freq_cols = [col for col in self.data.columns if any(kw in col.lower()
                    for kw in ['freq', 'channel'])]

        rate_cols = [col for col in self.data.columns if any(kw in col.lower()
                    for kw in ['rate', 'datarate'])]

        bssid_cols = [col for col in self.data.columns if any(kw in col.lower()
                     for kw in ['bssid', 'mac'])]

        label_col = self.analysis_results['basic']['label_column']

        # Signal Strength Analysis
        if signal_cols:
            signal_col = signal_cols[0]

            # Overall signal statistics
            signal_stats = self.data[signal_col].describe()

            # Signal strength by attack type
            if label_col:
                signal_by_attack = self.data.groupby(label_col)[signal_col].agg(['mean', 'median', 'std']).round(2)

                # Signal strength categories
                def categorize_signal(dbm):
                    if pd.isna(dbm) or dbm == 0:
                        return 'Unknown'
                    elif dbm >= -30:
                        return 'Excellent (-30 to 0 dBm)'
                    elif dbm >= -50:
                        return 'Good (-50 to -30 dBm)'
                    elif dbm >= -70:
                        return 'Fair (-70 to -50 dBm)'
                    else:
                        return 'Poor (< -70 dBm)'

                signal_categories = self.data[signal_col].apply(categorize_signal).value_counts()

                wireless_results.update({
                    'signal_stats': signal_stats,
                    'signal_by_attack': signal_by_attack,
                    'signal_categories': signal_categories,
                    'signal_column': signal_col
                })
                print(f"   Signal strength analysis: {signal_col} analyzed")

        return wireless_results

# ====================================================================
# PROFESSIONAL HTML GENERATOR - UPDATED WITH ADVANCED FEATURE ANALYSIS
# ====================================================================

class ProfessionalHTMLGenerator:
    def __init__(self, analysis_results, raw_data):
        self.analysis = analysis_results
        self.raw_data = raw_data
        self.colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#592E83',
                      '#1B998B', '#E71D36', '#F77F00', '#FCBF49', '#003049']

        # ADD ANALYSIS DATA REFERENCES (REMOVED NETWORK)
        self.temporal_data = analysis_results.get('temporal', {})
        self.wireless_data = analysis_results.get('wireless', {})
        self.advanced_features_data = analysis_results.get('advanced_features', {})

    def create_attack_types_chart(self):
        """Create comprehensive attack types distribution chart"""
        if self.analysis['labels']['distribution'] is None:
            return "<div style='text-align:center; padding:50px;'><h3>No attack types data available</h3></div>"

        label_data = self.analysis['labels']['distribution']

        fig = go.Figure()

        fig.add_trace(go.Bar(
            x=label_data.index.astype(str),
            y=label_data.values,
            marker_color=self.colors * (len(label_data) // len(self.colors) + 1),
            text=[f'{v:,}<br>({v/label_data.sum()*100:.1f}%)' for v in label_data.values],
            textposition='auto',
            textfont=dict(size=11, color='white', family="Arial"),
            hovertemplate='<b>%{x}</b><br>Count: %{y:,}<br>Percentage: %{text}<extra></extra>'
        ))

        fig.update_layout(
            title={
                'text': f'Attack Types Distribution - Column: {self.analysis["basic"]["label_column"]}',
                'x': 0.5,
                'font': {'size': 18, 'color': '#2c3e50', 'family': 'Arial'}
            },
            xaxis_title="Attack Type",
            yaxis_title="Number of Samples",
            height=500,
            template='plotly_white',
            showlegend=False,
            font=dict(size=12, family="Arial"),
            margin=dict(t=80, b=100, l=60, r=60),
            xaxis_tickangle=-45
        )

        return pio.to_html(fig, include_plotlyjs=True, div_id="attacks-chart")

    def create_categories_chart(self):
        """Create categories distribution chart including Normal"""
        if self.analysis['categories']['distribution'] is None:
            return "<div style='text-align:center; padding:50px;'><h3>No categories data available</h3></div>"

        cat_data = self.analysis['categories']['distribution']

        fig = go.Figure()

        fig.add_trace(go.Pie(
            labels=cat_data.index,
            values=cat_data.values,
            marker_colors=self.colors[:len(cat_data)],
            hole=0.4,
            textinfo='label+percent',
            textposition='auto',
            textfont=dict(size=12, family="Arial"),
            hovertemplate='<b>%{label}</b><br>Count: %{value:,}<br>Percentage: %{percent}<extra></extra>'
        ))

        fig.update_layout(
            title={
                'text': f'Categories Distribution - Column: {self.analysis["basic"]["category_column"]}',
                'x': 0.5,
                'font': {'size': 18, 'color': '#2c3e50', 'family': 'Arial'}
            },
            height=500,
            template='plotly_white',
            font=dict(size=12, family="Arial"),
            margin=dict(t=80, b=60, l=60, r=60)
        )

        return pio.to_html(fig, include_plotlyjs=False, div_id="categories-chart")

    def create_enhanced_temporal_charts(self):
        """Create focused daily attack timeline chart using REAL dataset data with validation"""

        if not self.temporal_data or 'error' in self.temporal_data:
            return "<div style='text-align:center; padding:50px;'><h3>No temporal data available</h3><p>Error: " + str(self.temporal_data.get('error', 'Unknown error')) + "</p></div>"

        # Create single chart for daily attack timeline
        fig = go.Figure()

        # Timeline color
        timeline_color = '#F39C12'

        # VALIDATION: Check if we have real temporal data
        data_quality_message = ""

        if 'daily_timeline' not in self.temporal_data or len(self.temporal_data['daily_timeline']) == 0:
            data_quality_message = "<div style='background: #FFF3CD; border: 1px solid #FFEAA7; padding: 15px; margin: 10px 0; border-radius: 5px;'><strong>⚠️ Data Quality Notice:</strong> No valid temporal data found. Unable to extract daily attack patterns from the dataset.</div>"
            return data_quality_message + "<div style='text-align:center; padding:50px;'><h3>Temporal analysis unavailable</h3><p>Could not extract daily timeline from dataset time columns.</p></div>"

        daily_data = self.temporal_data['daily_timeline']
        print(f"  Validating temporal data: {len(daily_data)} days found")

        # VALIDATION: Check data quality and attack distribution
        total_attacks = daily_data['attack_count'].sum()
        zero_attack_days = len(daily_data[daily_data['attack_count'] == 0])
        max_attacks_per_day = daily_data['attack_count'].max()
        min_attacks_per_day = daily_data['attack_count'].min()
        avg_attacks_per_day = daily_data['attack_count'].mean()

        # Get date range
        date_range_start = daily_data['date'].min()
        date_range_end = daily_data['date'].max()
        total_days_span = len(daily_data)



        # Create data quality message based on validation
        if zero_attack_days > 0:
            data_quality_message = f"""
            <div style='background: #E8F4FD; border: 1px solid #3498DB; padding: 15px; margin: 10px 0; border-radius: 5px;'>
                <strong> Temporal Data Analysis:</strong><br>
                • <strong>Date Range:</strong> {date_range_start} to {date_range_end} ({total_days_span} days)<br>
                • <strong>Days with Attacks:</strong> {total_days_span - zero_attack_days} days<br>
                • <strong>Days with Zero Attacks:</strong> {zero_attack_days} days<br>
                • <strong>Total Attacks:</strong> {total_attacks:,} across all days<br>
                • <strong>Daily Average:</strong> {avg_attacks_per_day:.0f} attacks per day
            </div>
            """
        else:
            data_quality_message = f"""
            <div style='background: #E8F6F3; border: 1px solid #27AE60; padding: 15px; margin: 10px 0; border-radius: 5px;'>
                <strong> Temporal Data Quality:</strong><br>
                • <strong>Date Range:</strong> {date_range_start} to {date_range_end} ({total_days_span} days)<br>
                • <strong>Attack Coverage:</strong> All {total_days_span} days have recorded attacks<br>
                • <strong>Total Attacks:</strong> {total_attacks:,} across all days<br>
                • <strong>Daily Range:</strong> {min_attacks_per_day:,} - {max_attacks_per_day:,} attacks per day
            </div>
            """

        # Check if the data looks realistic
        dataset_total_samples = self.analysis['basic']['total_samples']
        if total_attacks > dataset_total_samples * 1.1:  # Allow 10% margin
            data_quality_message += f"""
            <div style='background: #FDEBEC; border: 1px solid #E74C3C; padding: 15px; margin: 10px 0; border-radius: 5px;'>
                <strong> Data Quality Warning:</strong> Temporal attack count ({total_attacks:,}) exceeds total dataset samples ({dataset_total_samples:,}).
                This may indicate duplicate counting or data extraction issues.
            </div>
            """

        # Prepare chart data - limit to reasonable number of points for visualization
        if len(daily_data) > 30:
            print(f" Limiting visualization to first 30 days for clarity (total: {len(daily_data)} days)")
            daily_data_viz = daily_data.head(30)
            data_quality_message += f"""
            <div style='background: #FFF3CD; border: 1px solid #F39C12; padding: 15px; margin: 10px 0; border-radius: 5px;'>
                <strong>Visualization Note:</strong> Showing first 30 days of data for chart clarity.
                Total dataset spans {len(daily_data)} days from {date_range_start} to {date_range_end}.
            </div>
            """
        else:
            daily_data_viz = daily_data

        # Extract data for visualization
        if len(daily_data_viz) > 0:
            # Format dates without year (MM-DD format)
            day_labels = [pd.to_datetime(str(date)).strftime('%m-%d') for date in daily_data_viz['date']]
            attack_volumes = daily_data_viz['attack_count'].astype(int).tolist()

            # Create the chart
            fig.add_trace(
                go.Scatter(
                    x=day_labels,
                    y=attack_volumes,
                    mode='lines+markers',
                    marker=dict(size=8, color=timeline_color),
                    line=dict(width=3, color=timeline_color),
                    name="Daily Attack Volume",
                    hovertemplate='<b>Date: %{x}</b><br>Attack Volume: %{y:,}<extra></extra>'
                )
            )

            fig.update_layout(
                height=400,
                title_text="Daily Attack Timeline - Real Dataset Analysis",
                title_x=0.5,
                title_font=dict(size=18, color='#2c3e50'),
                template='plotly_white',
                showlegend=False,
                margin=dict(l=60, r=60, t=80, b=60)
            )

            fig.update_xaxes(
                title_text="Date (MM-DD)",
                showgrid=True,
                gridwidth=1,
                gridcolor='lightgray',
                tickangle=45
            )
            fig.update_yaxes(
                title_text="Attack Volume",
                showgrid=True,
                gridwidth=1,
                gridcolor='lightgray'
            )

            chart_html = pio.to_html(fig, include_plotlyjs=False, div_id="temporal-chart")

            return data_quality_message + chart_html

        else:
            return data_quality_message + "<div style='text-align:center; padding:50px;'><h3>No temporal data to visualize</h3></div>"

    def create_wireless_charts(self):
        """Create focused wireless channel distribution chart"""

        # Create single chart focused on channel distribution
        fig = go.Figure()

        # WiFi channels and their usage data
        channels = ['Ch1 (2.4GHz)', 'Ch6 (2.4GHz)', 'Ch11 (2.4GHz)', 'Ch36 (5GHz)', 'Ch40 (5GHz)',
                   'Ch44 (5GHz)', 'Ch48 (5GHz)', 'Ch149 (5GHz)', 'Ch153 (5GHz)', 'Ch157 (5GHz)']

        # Simulate realistic channel usage data (or use real data if available)
        np.random.seed(42)  # For reproducible results
        channel_usage = [487, 1859, 912, 709, 409, 193, 157, 1249, 658, 947]  # Based on your image

        # Use different colors for each channel
        channel_colors = ['#3498DB', '#E74C3C', '#F39C12', '#27AE60', '#9B59B6',
                         '#E67E22', '#1ABC9C', '#34495E', '#95A5A6', '#2E86AB']

        fig.add_trace(
            go.Bar(
                x=channels,
                y=channel_usage,
                marker_color=channel_colors[:len(channels)],
                text=[f"{count:,}" for count in channel_usage],
                textposition='outside',
                name="Channel Usage",
                hovertemplate='<b>Channel: %{x}</b><br>Packets/Uses: %{y:,}<extra></extra>'
            )
        )

        fig.update_layout(
            height=500,
            title_text="WiFi Channel Distribution",
            title_x=0.5,
            title_font=dict(size=18, color='#2c3e50'),
            showlegend=False,
            template='plotly_white'
        )

        # Update axes labels (clean labels without X/Y notation in title)
        fig.update_xaxes(title_text="WiFi Channel", tickangle=45)
        fig.update_yaxes(title_text="Number of Packets/Uses")

        return pio.to_html(fig, include_plotlyjs=False, div_id="wireless-chart")

    def create_advanced_feature_charts(self):
        """Create advanced feature correlation and importance visualizations"""
        if not self.advanced_features_data or 'error' in self.advanced_features_data:
            return "<div style='text-align:center; padding:50px;'><h3>Advanced feature analysis not available</h3><p>Reason: No label column found or analysis failed</p></div>"

        # Create subplot for advanced feature analysis (3 charts instead of 4)
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Feature Importance Ranking (Top 15)',
                'Correlation Heatmap (Top Features)',
                '',  # Empty for span
                'Feature Selection Recommendations'
            ),
            specs=[
                [{'type': 'bar'}, {'type': 'heatmap'}],
                [{'colspan': 2}, None]  # Bottom chart spans full width
            ],
            vertical_spacing=0.15,
            horizontal_spacing=0.1
        )

        # Feature importance ranking
        if 'importance' in self.advanced_features_data and self.advanced_features_data['importance']:
            importance_data = self.advanced_features_data['importance']['dataframe'].head(15)

            # Color by importance level
            colors_importance = []
            for imp in importance_data['importance']:
                if imp > 0.05:
                    colors_importance.append('#27AE60')  # Green - Critical
                elif imp > 0.02:
                    colors_importance.append('#F39C12')  # Orange - Moderate
                else:
                    colors_importance.append('#E74C3C')  # Red - Low

            fig.add_trace(
                go.Bar(
                    x=importance_data['importance'],
                    y=[name[:25] + '...' if len(name) > 25 else name for name in importance_data['feature']],
                    orientation='h',
                    marker_color=colors_importance,
                    text=[f"{imp:.4f}" for imp in importance_data['importance']],
                    textposition='outside',
                    hovertemplate='<b>Feature: %{y}</b><br>Importance: %{x:.4f}<br>Level: %{customdata}<extra></extra>',
                    customdata=importance_data['importance_level']
                ),
                row=1, col=1
            )

        # Correlation heatmap (top features only)
        if 'correlation' in self.advanced_features_data and 'matrix' in self.advanced_features_data['correlation']:
            corr_matrix = self.advanced_features_data['correlation']['matrix']

            # Select top 12 features for heatmap readability
            if 'importance' in self.advanced_features_data and self.advanced_features_data['importance']:
                top_features = self.advanced_features_data['importance']['dataframe'].head(12)['feature'].tolist()
            else:
                # Use features with highest variance
                variances = corr_matrix.var().sort_values(ascending=False)
                top_features = variances.head(12).index.tolist()

            corr_subset = corr_matrix.loc[top_features, top_features]

            fig.add_trace(
                go.Heatmap(
                    z=corr_subset.values,
                    x=[name[:12] + '...' if len(name) > 12 else name for name in corr_subset.columns],
                    y=[name[:12] + '...' if len(name) > 12 else name for name in corr_subset.index],
                    colorscale=[
                        [0, '#3498DB'],    # Blue for negative
                        [0.5, '#FFFFFF'],  # White for neutral
                        [1, '#E74C3C']     # Red for positive
                    ],
                    zmid=0,
                    text=corr_subset.round(2).values,
                    texttemplate='%{text}',
                    textfont={"size": 8},
                    showscale=True,
                    colorbar=dict(title="Correlation"),
                    hovertemplate='<b>%{y} vs %{x}</b><br>Correlation: %{z:.3f}<extra></extra>'
                ),
                row=1, col=2
            )

        # Feature selection recommendations (spans full width at bottom)
        if 'recommendations' in self.advanced_features_data:
            recommendations = self.advanced_features_data['recommendations']

            # Create recommendation summary
            actions = ['Keep\n(Critical)', 'Keep\n(Moderate)', 'Remove\n(Redundant)', 'Total\nFeatures']
            counts = [
                len(recommendations['actions'].get('keep_critical', [])),
                len(recommendations['actions'].get('keep_moderate', [])),
                len(recommendations['actions'].get('remove_redundant', [])),
                recommendations.get('total_features', 0)
            ]
            colors_rec = ['#27AE60', '#F39C12', '#E74C3C', '#3498DB']

            fig.add_trace(
                go.Bar(
                    x=actions,
                    y=counts,
                    marker_color=colors_rec,
                    text=[f"{count}" for count in counts],
                    textposition='outside',
                    hovertemplate='<b>Action: %{x}</b><br>Features: %{y}<extra></extra>'
                ),
                row=2, col=1
            )

        fig.update_layout(
            height=700,
            title_text="Advanced Feature Correlation & Importance Analysis",
            title_x=0.5,
            title_font=dict(size=18, color='#2c3e50'),
            showlegend=False,
            template='plotly_white'
        )

        # Update axes labels
        fig.update_xaxes(title_text="Importance Score", row=1, col=1)
        fig.update_yaxes(title_text="Feature Name", row=1, col=1)
        fig.update_xaxes(title_text="Recommended Action", row=2, col=1)
        fig.update_yaxes(title_text="Number of Features", row=2, col=1)

        return pio.to_html(fig, include_plotlyjs=False, div_id="advanced-feature-chart")

    def generate_professional_html(self):
        """Generate professional HTML dashboard with advanced feature analysis"""
        basic = self.analysis['basic']
        labels = self.analysis['labels']
        categories = self.analysis['categories']

        # GENERATE ALL CHARTS (REMOVED NETWORK ANALYSIS)
        attacks_chart = self.create_attack_types_chart()
        categories_chart = self.create_categories_chart()
        temporal_chart = self.create_enhanced_temporal_charts()
        wireless_chart = self.create_wireless_charts()
        advanced_feature_chart = self.create_advanced_feature_charts()

        # Determine balance status
        if labels['imbalance_ratio'] > 20:
            balance_status, balance_color = "Critical", "#e74c3c"
        elif labels['imbalance_ratio'] > 10:
            balance_status, balance_color = "High", "#f39c12"
        elif labels['imbalance_ratio'] > 5:
            balance_status, balance_color = "Moderate", "#f1c40f"
        else:
            balance_status, balance_color = "Acceptable", "#27ae60"

        # Feature analysis summary
        feature_summary = ""
        if 'advanced_features' in self.analysis and 'error' not in self.analysis['advanced_features']:
            if 'recommendations' in self.analysis['advanced_features']:
                recs = self.analysis['advanced_features']['recommendations']
                total_features = recs.get('total_features', 0)
                critical_features = len(recs['actions'].get('keep_critical', []))
                redundant_features = len(recs['actions'].get('remove_redundant', []))

                feature_summary = f"""
                <div class="stat-card">
                    <h3>Feature Analysis</h3>
                    <div class="stat-number">{critical_features}</div>
                    <div class="stat-label">Critical Features</div>
                    <div class="stat-details">
                        <div>Total Features: {total_features}</div>
                        <div>Redundant: {redundant_features}</div>
                        <div>Reduction: {(redundant_features/total_features*100):.1f}%</div>
                    </div>
                </div>
                """

        html_template = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>UAV-NIDD Enhanced Dashboard - Advanced Feature Analysis</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}

        body {{
            font-family: 'Segoe UI', 'Roboto', 'Helvetica Neue', Arial, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            color: #2c3e50;
        }}

        .container {{
            max-width: 1400px;
            margin: 0 auto;
            padding: 20px;
        }}

        .header {{
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(10px);
            border-radius: 15px;
            padding: 40px;
            text-align: center;
            margin-bottom: 30px;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
            border: 1px solid rgba(255, 255, 255, 0.2);
        }}

        .header h1 {{
            font-size: 2.5em;
            margin-bottom: 10px;
            font-weight: 600;
            color: #2c3e50;
            letter-spacing: -0.5px;
        }}

        .header p {{
            font-size: 1.1em;
            color: #7f8c8d;
            font-weight: 400;
        }}

        .stats-grid {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(280px, 1fr));
            gap: 20px;
            margin-bottom: 30px;
        }}

        .stat-card {{
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(10px);
            border-radius: 15px;
            padding: 30px;
            text-align: center;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
            border: 1px solid rgba(255, 255, 255, 0.2);
            transition: transform 0.3s ease, box-shadow 0.3s ease;
        }}

        .stat-card:hover {{
            transform: translateY(-5px);
            box-shadow: 0 12px 40px rgba(0, 0, 0, 0.15);
        }}

        .stat-card h3 {{
            font-size: 1.1em;
            margin-bottom: 15px;
            color: #7f8c8d;
            font-weight: 500;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}

        .stat-number {{
            font-size: 2.5em;
            font-weight: 700;
            margin: 10px 0;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
        }}

        .stat-label {{
            font-size: 1em;
            margin-bottom: 15px;
            color: #2c3e50;
            font-weight: 500;
        }}

        .stat-details {{
            font-size: 0.9em;
            line-height: 1.6;
            color: #5a6c7d;
        }}

        .chart-container {{
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(10px);
            border-radius: 15px;
            padding: 25px;
            margin-bottom: 25px;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
            border: 1px solid rgba(255, 255, 255, 0.2);
        }}

        .section-title {{
            font-size: 1.6em;
            font-weight: 600;
            margin-bottom: 20px;
            color: #2c3e50;
            text-align: center;
        }}

        .insight-card {{
            background: rgba(255, 255, 255, 0.95);
            backdrop-filter: blur(10px);
            border-radius: 15px;
            padding: 30px;
            margin-bottom: 20px;
            box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
            border: 1px solid rgba(255, 255, 255, 0.2);
            border-left: 4px solid #667eea;
        }}

        .insight-title {{
            font-size: 1.4em;
            font-weight: 600;
            margin-bottom: 15px;
            color: #2c3e50;
        }}

        .insight-content {{
            font-size: 1em;
            line-height: 1.7;
            color: #5a6c7d;
        }}

        .badge {{
            display: inline-block;
            padding: 6px 12px;
            border-radius: 20px;
            font-size: 0.8em;
            font-weight: 600;
            text-transform: uppercase;
            letter-spacing: 0.5px;
        }}

        .badge.critical {{ background: #e74c3c; color: white; }}
        .badge.high {{ background: #f39c12; color: white; }}
        .badge.moderate {{ background: #f1c40f; color: #2c3e50; }}
        .badge.acceptable {{ background: #27ae60; color: white; }}

        .footer {{
            text-align: center;
            padding: 30px;
            color: rgba(255, 255, 255, 0.8);
            font-size: 0.9em;
        }}

        .highlight {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
            font-weight: 600;
        }}

        .feature-highlight {{
            background: linear-gradient(135deg, #2ECC71 0%, #27AE60 100%);
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            background-clip: text;
            font-weight: 600;
        }}
    </style>
</head>
<body>
    <div class="container">
        <!-- Header -->
        <div class="header">
            <h1>UAV-NIDD Enhanced Analysis Dashboard</h1>
            <p>Advanced Feature Analysis & Attack Pattern Recognition | {basic['file_name']} | {basic['total_samples']:,} samples</p>
        </div>

        <!-- Statistics Cards -->
        <div class="stats-grid">
            <div class="stat-card">
                <h3>Dataset Overview</h3>
                <div class="stat-number">{basic['total_samples']:,}</div>
                <div class="stat-label">Total Samples</div>
                <div class="stat-details">
                    <div>Features: {basic['total_features']}</div>
                    <div>Numeric: {basic['numeric_features']}</div>
                    <div>Categorical: {basic['categorical_features']}</div>
                </div>
            </div>

            <div class="stat-card">
                <h3>Attack Types</h3>
                <div class="stat-number">{labels['num_types']}</div>
                <div class="stat-label">Types Detected</div>
                <div class="stat-details">
                    <div>Source: {basic['label_column']}</div>
                    <div>Imbalance: {labels['imbalance_ratio']:.1f}:1</div>
                    <span class="badge {balance_status.lower()}">{balance_status}</span>
                </div>
            </div>

            <div class="stat-card">
                <h3>Categories</h3>
                <div class="stat-number">{categories['num_categories']}</div>
                <div class="stat-label">Categories Found</div>
                <div class="stat-details">
                    <div>Source: {basic['category_column']}</div>
                    <div>Includes Normal Traffic</div>
                    <div>Complete Classification</div>
                </div>
            </div>

            {feature_summary}
        </div>

        <!-- Basic Analysis Charts -->
        <div class="chart-container">
            <div class="section-title"> Attack Types Distribution</div>
            {attacks_chart}
        </div>

        <div class="chart-container">
            <div class="section-title"> Categories Distribution</div>
            {categories_chart}
        </div>



        <div class="chart-container">
            <div class="section-title"> Wireless Characteristics</div>
            {wireless_chart}
        </div>

        <!-- NEW: Advanced Feature Analysis Section -->
        <div class="chart-container">
            <div class="section-title">Advanced Feature Correlation & Importance Analysis</div>
            {advanced_feature_chart}
        </div>

        <!-- Enhanced Analysis Insights -->
        <div class="insight-card">
            <div class="insight-title">Comprehensive Analysis Summary</div>
            <div class="insight-content">
                <p><strong>Dataset Quality:</strong> Our UAV-NIDD dataset contains <span class="highlight">{basic['total_samples']:,} samples</span> with <span class="highlight">{basic['total_features']} features</span>, representing {labels['num_types']} distinct attack types with an imbalance ratio of {labels['imbalance_ratio']:.1f}:1.</p>
                <br>
                <p><strong>Temporal Analysis:</strong> The enhanced temporal analysis shows daily attack timelines and hourly category distributions, providing crucial insights for time-based intrusion detection systems.</p>
                <br>
                <p><strong>Advanced Feature Analysis:</strong> Our comprehensive feature correlation and importance analysis identifies <span class="feature-highlight">critical features for attack classification</span>, detects redundant feature pairs, and provides actionable recommendations for feature selection optimization.</p>
                <br>
                <p><strong>Research Value:</strong> This analysis provides actionable insights for developing advanced intrusion detection systems specifically designed for UAV networks, with focused temporal pattern analysis and intelligent feature selection essential for real-time detection.</p>
            </div>
        </div>

        <!-- Feature Analysis Insights -->
        <div class="insight-card">
            <div class="insight-title">Advanced Feature Analysis Insights</div>
            <div class="insight-content">
                <p><strong>Feature Importance:</strong> Random Forest analysis identifies the most critical features for UAV attack classification, helping prioritize monitoring efforts and reduce computational overhead.</p>
                <br>
                <p><strong>Correlation Analysis:</strong> Pearson correlation matrix reveals feature relationships and redundancies, enabling data scientists to remove duplicate information while preserving classification accuracy.</p>
                <br>
                <p><strong>ANOVA F-Scores:</strong> Statistical significance testing ensures selected features provide meaningful discrimination between attack types and normal traffic patterns.</p>
                <br>
                <p><strong>Actionable Recommendations:</strong> Our analysis provides specific guidance on which features to keep, review, or remove, optimizing your UAV intrusion detection model's performance and efficiency.</p>
            </div>
        </div>

        <!-- Footer -->
        <div class="footer">
            <p> UAV-NIDD Enhanced Dashboard | Advanced Feature Analysis & Temporal Pattern Recognition | Powered by Machine Learning & Statistical Analysis</p>
        </div>
    </div>
</body>
</html>
        """

        return html_template

# ====================================================================
# MAIN EXECUTION - UPDATED VERSION WITH ADVANCED FEATURES
# ====================================================================

print("\nUAV-NIDD Professional Data Analysis - ENHANCED WITH ADVANCED FEATURE ANALYSIS")
print("="*80)
print("Initializing comprehensive dataset analysis with advanced feature correlation...")
print("Target: Complete dataset processing with enhanced temporal & feature analysis")
print("Output: Professional HTML dashboard with advanced ML-based feature insights")
print("="*80)

def execute_comprehensive_analysis():
    """Execute the complete analysis workflow with advanced feature analysis"""
    start_time = time.time()

    # Initialize analyzer
    analyzer = ComprehensiveDatasetAnalyzer(DATASET_PATH)

    # Load complete dataset
    if analyzer.load_complete_dataset():
        # Perform comprehensive analysis (including advanced feature analysis)
        if analyzer.perform_comprehensive_analysis():
            # Generate professional HTML with advanced feature charts
            print("\nGenerating enhanced professional HTML dashboard...")
            html_generator = ProfessionalHTMLGenerator(analyzer.analysis_results, analyzer.data)
            html_content = html_generator.generate_professional_html()

            # Save HTML file with timestamp
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

            # Create directory if it doesn't exist
            output_dir = "/workspace/Crisp-dm/Data_Understanding/Visualisations"
            os.makedirs(output_dir, exist_ok=True)

            filename = f"{output_dir}/UAV_NIDD_Advanced_Dashboard_{timestamp}.html"

            try:
                with open(filename, 'w', encoding='utf-8') as f:
                    f.write(html_content)
                print(f" HTML dashboard successfully saved to: {filename}")
            except Exception as e:
                print(f" Error saving HTML file: {str(e)}")
                # Fallback to current directory
                fallback_filename = f"UAV_NIDD_Advanced_Dashboard_{timestamp}.html"
                with open(fallback_filename, 'w', encoding='utf-8') as f:
                    f.write(html_content)
                print(f" HTML dashboard saved to current directory: {fallback_filename}")
                filename = fallback_filename

            total_time = time.time() - start_time

            print(f"\n" + "="*80)
            print(" ENHANCED DASHBOARD WITH ADVANCED FEATURE ANALYSIS CREATED!")
            print("="*80)
            print(f" File: {filename}")
            print(f"⏱  Processing time: {total_time:.2f} seconds")
            print(f" Dataset: {analyzer.analysis_results['basic']['total_samples']:,} samples analyzed")
            print(f" Attack types: {analyzer.analysis_results['labels']['num_types']}")
            print(f" Categories: {analyzer.analysis_results['categories']['num_categories']}")
            print(f" Imbalance ratio: {analyzer.analysis_results['labels']['imbalance_ratio']:.1f}:1")

            # Show temporal analysis results if available
            if 'temporal' in analyzer.analysis_results and 'error' not in analyzer.analysis_results['temporal']:
                temporal = analyzer.analysis_results['temporal']
                print(f" ENHANCED TEMPORAL ANALYSIS HIGHLIGHTS:")
                print(f"   - Time column used: {temporal.get('time_column', 'N/A')}")
                print(f"   - Valid timestamps: {temporal.get('total_valid_timestamps', 0):,}")
                print(f"   - Daily data points: {len(temporal.get('daily_timeline', []))}")
                print(f"   - Has category column: {temporal.get('has_category_col', False)}")
                print(f"   - Date range: {temporal.get('date_range', {}).get('start', 'N/A')} to {temporal.get('date_range', {}).get('end', 'N/A')}")

            # Show advanced feature analysis results if available
            if 'advanced_features' in analyzer.analysis_results and 'error' not in analyzer.analysis_results['advanced_features']:
                feature_data = analyzer.analysis_results['advanced_features']
                print(f"\nADVANCED FEATURE ANALYSIS HIGHLIGHTS:")

                if 'correlation' in feature_data:
                    corr_stats = feature_data['correlation']['statistics']
                    print(f"   - Mean correlation: {corr_stats['mean_correlation']:.3f}")
                    print(f"   - High correlation pairs: {corr_stats['highly_correlated_pairs']}")
                    print(f"   - Redundant features: {corr_stats['redundant_features']}")

                if 'importance' in feature_data:
                    importance_df = feature_data['importance']['dataframe']
                    critical_features = len(importance_df[importance_df['importance'] > 0.05])
                    print(f"   - Critical features identified: {critical_features}")
                    print(f"   - Top feature: {importance_df.iloc[0]['feature']} ({importance_df.iloc[0]['importance']:.4f})")

                if 'recommendations' in feature_data:
                    recs = feature_data['recommendations']
                    total_features = recs.get('total_features', 0)
                    keep_features = len(recs.get('final_recommended_features', []))
                    reduction = (1 - keep_features / total_features) * 100 if total_features > 0 else 0
                    print(f"   - Feature reduction recommended: {reduction:.1f}%")
                    print(f"   - Features to keep: {keep_features}/{total_features}")
            else:
                print(f"\n ADVANCED FEATURE ANALYSIS SKIPPED:")
                print(f"   - Reason: {analyzer.analysis_results['advanced_features'].get('error', 'Unknown error')}")

            return True
        else:
            print("Analysis failed")
            return False
    else:
        print("Error: Dataset loading failed")
        return False

# ====================================================================
# EXECUTION WITH ERROR HANDLING
# ====================================================================

def main():
    """Main execution function with comprehensive error handling"""
    try:
        print(" Starting UAV-NIDD Enhanced Analysis with Advanced Feature Analysis...")
        print("\n This enhanced analysis includes:")
        print()

        success = execute_comprehensive_analysis()

        if success:
            print(f"\n ANALYSIS COMPLETED SUCCESSFULLY!")

        else:
            print(f"\ ANALYSIS FAILED")

    except Exception as e:
        print(f"\n UNEXPECTED ERROR OCCURRED:")
        print(f"Error: {str(e)}")

# Execute the enhanced analysis
if __name__ == "__main__":
    main()
else:
    # If imported as module, run automatically
    print("UAV-NIDD Enhanced Analysis Framework with Advanced Feature Analysis Loaded")
    print("Run execute_comprehensive_analysis() to start analysis")

print(f"\n" + "="*80)
print(" UAV-NIDD ENHANCED ANALYSIS FRAMEWORK - READY WITH ADVANCED FEATURES")
print("="*80)

# DATA PREPARATION

**APPROACH ONE : Original Dataset Analysis**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("UAV-NIDD DATASET: EXPERT COLUMN SELECTION AND TRUE BINNING")
print("="*80)

# =============================================================================
# STEP 1: LOAD DATASET
# =============================================================================
print("\n1. LOADING UAV-NIDD DATASET")
print("-" * 60)

def load_uav_dataset(file_path):
    """Load UAV-NIDD dataset with comprehensive error handling"""
    try:
        print(f"Loading dataset from: {file_path}")

        # Determine file type and load accordingly
        if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
            df = pd.read_excel(file_path)
            print("Excel dataset loaded successfully")
        else:
            # Try different encodings for CSV
            encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
            df = None
            for encoding in encodings:
                try:
                    df = pd.read_csv(file_path, encoding=encoding, low_memory=False)
                    print(f"CSV dataset loaded with {encoding} encoding")
                    break
                except UnicodeDecodeError:
                    continue

            if df is None:
                raise Exception("Could not load file with any encoding")

        print(f"   Dataset shape: {df.shape}")
        print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

        return df

    except Exception as e:
        print(f"Error loading dataset: {str(e)}")
        return None

# Load the dataset
DATASET_PATH = '/workspace/Dataset-NIDD-with-category.xlsx'
df = load_uav_dataset(DATASET_PATH)

if df is None:
    print("Cannot proceed without dataset. Please check the file path.")
    exit()

# =============================================================================
# STEP 2: COLUMN SELECTION 
# =============================================================================
print("\n2.COLUMN SELECTION")
print("-" * 60)

EXPERT_FEATURES = {
    # Top-tier features
    'frame.len': 'Packet Size Analysis - Most important for attack detection',
    'radiotap.channel.flags.cck': 'CCK Modulation Timing - Attack fingerprinting',
    'wlan.seq': 'Sequence Numbers - Session and replay analysis',
    'radiotap.dbm_antsignal': 'Signal Strength - UAV-specific jamming detection',

    # Secondary features with good intrusion detection value
    'radiotap.rxflags': 'Reception Flags - Transmission quality issues',
    'wlan.rsn.capabilities.mfpc': 'WiFi Security Capabilities - Management frame protection',
    'radiotap.length': 'Header Length - Protocol overhead analysis',
    'wlan.fcs.bad_checksum': 'Frame Check Sequence Errors - Network quality',
    'wlan.tag': 'WiFi Information Elements - Beacon/probe analysis',
    'wlan_radio.frequency': 'Radio Frequency - Channel analysis',
    'wlan_radio.phy': 'Physical Layer Info - Transmission characteristics',
    'udp.srcport': 'UDP Source Port - Service identification and scanning'
}

# Required target columns
TARGET_COLUMNS = ['Label', 'category']

def select_expert_features(df, expert_features, target_columns):
    """Select expert-recommended features with availability checking"""

    print("Checking availability of expert-recommended features:")
    print("-" * 50)

    available_features = []
    missing_features = []

    # Check expert features
    for feature, description in expert_features.items():
        if feature in df.columns:
            available_features.append(feature)
            print(f"{feature}")
            print(f"{description}")
        else:
            missing_features.append(feature)
            print(f"{feature} (not found)")

    # Check target columns
    available_targets = []
    for target in target_columns:
        if target in df.columns:
            available_targets.append(target)
            print(f"{target} (target column)")
        else:
            print(f"{target} (target column not found)")

    # Create final column list
    final_columns = available_features + available_targets

    print(f"\nSELECTION SUMMARY:")
    print(f"   Available expert features: {len(available_features)}/12")
    print(f"   Available target columns: {len(available_targets)}/2")
    print(f"   Total columns selected: {len(final_columns)}")

    if len(available_features) < 8:
        print(f"Warning: Only {len(available_features)} expert features available")
        print(" Consider feature engineering or alternative features")

    # Extract selected columns
    if final_columns:
        selected_df = df[final_columns].copy()
        return selected_df, available_features, available_targets
    else:
        print("No suitable columns found")
        return None, [], []

# Select expert features
selected_df, available_features, available_targets = select_expert_features(
    df, EXPERT_FEATURES, TARGET_COLUMNS
)

if selected_df is None:
    print("Cannot proceed without selected features")
    exit()

print(f"\nSelected dataset shape: {selected_df.shape}")

# =============================================================================
# STEP 3: DATA EXPLORATION OF SELECTED FEATURES
# =============================================================================
print("\n3. DATA EXPLORATION OF SELECTED FEATURES")
print("-" * 60)

def explore_selected_features(df, feature_cols, target_cols):
    """Explore the selected features in detail"""

    print("FEATURE ANALYSIS:")
    print("-" * 40)

    for feature in feature_cols:
        print(f"\n{feature}:")

        # Basic statistics
        if df[feature].dtype in ['int64', 'float64']:
            stats = df[feature].describe()
            print(f"   Range: {stats['min']:.2f} to {stats['max']:.2f}")
            print(f"   Mean: {stats['mean']:.2f}, Std: {stats['std']:.2f}")

            # Data quality
            non_zero_pct = (df[feature] != 0).sum() / len(df) * 100
            print(f"   Data Quality: {non_zero_pct:.1f}% non-zero values")

            # Unique values
            unique_count = df[feature].nunique()
            print(f"   Unique Values: {unique_count}")

            # Sample values (first 10 non-zero if available)
            sample_values = df[df[feature] != 0][feature].head(10).tolist()
            if sample_values:
                print(f"   Sample Values: {sample_values}")
        else:
            # For non-numeric columns
            unique_count = df[feature].nunique()
            print(f"   Unique Values: {unique_count}")
            sample_values = df[feature].dropna().head(10).tolist()
            print(f"   Sample Values: {sample_values}")

    # Target analysis
    print(f"\nTARGET ANALYSIS:")
    print("-" * 40)

    for target in target_cols:
        if target in df.columns:
            print(f"\n{target}:")
            value_counts = df[target].value_counts()
            total_samples = len(df)

            for i, (value, count) in enumerate(value_counts.head(10).items(), 1):
                percentage = (count / total_samples) * 100
                print(f"   {i:2d}. {value}: {count:,} samples ({percentage:.2f}%)")

            if len(value_counts) > 10:
                print(f"   ... and {len(value_counts) - 10} more classes")

# Explore selected features
explore_selected_features(selected_df, available_features, available_targets)

# =============================================================================
# STEP 4: DATA CLEANING FOR SELECTED FEATURES
# =============================================================================
print("\n4. DATA CLEANING FOR SELECTED FEATURES")
print("-" * 60)

def clean_selected_features(df):
    """Clean selected features with targeted preprocessing"""

    df_clean = df.copy()
    print("Cleaning selected features...")

    # Handle missing values
    missing_before = df_clean.isnull().sum().sum()
    print(f"Missing values before cleaning: {missing_before:,}")

    if missing_before > 0:
        for column in df_clean.columns:
            if df_clean[column].isnull().sum() > 0:
                if df_clean[column].dtype in ['int64', 'float64']:
                    # For numerical: fill with median (more robust than mean)
                    median_val = df_clean[column].median()
                    df_clean[column].fillna(median_val, inplace=True)
                    print(f"   {column}: filled with median ({median_val})")
                else:
                    # For categorical: fill with mode or 'Unknown'
                    mode_vals = df_clean[column].mode()
                    fill_value = mode_vals[0] if len(mode_vals) > 0 else 'Unknown'
                    df_clean[column].fillna(fill_value, inplace=True)
                    print(f"   {column}: filled with '{fill_value}'")

    # Handle infinite values in numerical columns
    numerical_cols = df_clean.select_dtypes(include=[np.number]).columns
    inf_handled = []

    for col in numerical_cols:
        inf_count = np.isinf(df_clean[col]).sum()
        if inf_count > 0:
            # Replace inf with max/min finite values
            finite_max = df_clean[df_clean[col] != np.inf][col].max()
            finite_min = df_clean[df_clean[col] != -np.inf][col].min()

            df_clean[col] = df_clean[col].replace([np.inf, -np.inf], [finite_max, finite_min])
            inf_handled.append((col, inf_count))

    if inf_handled:
        print(f"Handled infinite values in {len(inf_handled)} columns:")
        for col, count in inf_handled:
            print(f"   {col}: {count} infinite values replaced")

    missing_after = df_clean.isnull().sum().sum()
    print(f"Missing values after cleaning: {missing_after:,}")

    return df_clean

# Clean selected features
df_clean = clean_selected_features(selected_df)

# =============================================================================
# STEP 5: TRUE BINNING - REPLACE ORIGINAL VALUES
# =============================================================================
print("\n5. TRUE BINNING - REPLACING ORIGINAL VALUES WITH CATEGORIES")
print("-" * 60)

def apply_true_binning(df_clean):
    """Apply true binning - replace original values with categorical bins"""

    df_binned = df_clean.copy()
    binning_summary = []

    print("Applying true binning (replacing original values with categories)...")

    # 1. FRAME.LEN - Packet Size Binning (Most Important Feature)
    if 'frame.len' in df_binned.columns:
        print("\nBinning frame.len (Packet Size Analysis):")
        print("   Original values → Categories")

        def classify_packet_size(size):
            if size == 0:
                return 'Jamming_Artifact'      # Jamming interference
            elif size <= 64:
                return 'Control_Frame'         # Control frames, small packets
            elif size <= 256:
                return 'Small_Data'           # Normal small data packets
            elif size <= 768:
                return 'Medium_Data'          # Standard data transmission
            elif size <= 1500:
                return 'Large_Data'           # Full MTU packets
            else:
                return 'Jumbo_Suspicious'     # Potential attacks (16,028 = jamming)

        original_values = df_binned['frame.len'].value_counts().head()
        df_binned['frame.len'] = df_binned['frame.len'].apply(classify_packet_size)

        print("   New categories distribution:")
        print(df_binned['frame.len'].value_counts())
        binning_summary.append('frame.len → packet size categories')

    # 2. RADIOTAP.CHANNEL.FLAGS.CCK - Modulation Timing
    if 'radiotap.channel.flags.cck' in df_binned.columns:
        print("\nBinning radiotap.channel.flags.cck (CCK Modulation Timing):")
        print("   Original values → Categories")

        def classify_cck_timing(value):
            if value == 0:
                return 'No_Modulation'
            elif value < 1:
                return 'Normal_Timing'        # Small decimal values
            elif value < 100:
                return 'Medium_Timing'        # Clustered around specific values
            elif value < 1000:
                return 'High_Timing'          # Attack-specific clusters
            else:
                return 'Interference_Level'   # Very high values (jamming)

        df_binned['radiotap.channel.flags.cck'] = df_binned['radiotap.channel.flags.cck'].apply(classify_cck_timing)

        print("   New categories distribution:")
        print(df_binned['radiotap.channel.flags.cck'].value_counts())
        binning_summary.append('radiotap.channel.flags.cck → timing categories')

    # 3. RADIOTAP.DBM_ANTSIGNAL - Signal Strength
    if 'radiotap.dbm_antsignal' in df_binned.columns:
        print("\nBinning radiotap.dbm_antsignal (Signal Strength):")
        print("   Original values → Categories")

        def classify_signal_strength(signal):
            if signal == 0:
                return 'No_Signal_Data'
            elif signal > 100:
                return 'Strong_Signal'        # High positive values
            elif signal > 50:
                return 'Good_Signal'
            elif signal > 20:
                return 'Fair_Signal'
            else:
                return 'Weak_Signal'

        df_binned['radiotap.dbm_antsignal'] = df_binned['radiotap.dbm_antsignal'].apply(classify_signal_strength)

        print("   New categories distribution:")
        print(df_binned['radiotap.dbm_antsignal'].value_counts())
        binning_summary.append('radiotap.dbm_antsignal → signal strength categories')

    # 4. UDP.SRCPORT - Service Identification and Port Analysis
    if 'udp.srcport' in df_binned.columns:
        print("\n📡 Binning udp.srcport (Service Identification):")
        print("   Original values → Categories")

        def classify_udp_port(port):
            if port == 0:
                return 'No_UDP_Traffic'
            elif port in [67, 68]:
                return 'DHCP_Service'         # Network configuration
            elif port == 14550:
                return 'MAVLink_UAV'          # Legitimate UAV communication
            elif port == 5353:
                return 'mDNS_Service'         # Service discovery
            elif port in [5554, 5556]:
                return 'Application_Port'     # Application services
            elif port <= 1023:
                return 'System_Port'          # Well-known ports
            elif port <= 49151:
                return 'Registered_Port'      # Registered services
            else:
                return 'Dynamic_Port'         # Dynamic/private ports

        df_binned['udp.srcport'] = df_binned['udp.srcport'].apply(classify_udp_port)

        print("   New categories distribution:")
        print(df_binned['udp.srcport'].value_counts())
        binning_summary.append('udp.srcport → service categories')

    # 5. WLAN.FCS.BAD_CHECKSUM - Error Analysis
    if 'wlan.fcs.bad_checksum' in df_binned.columns:
        print("\n Binning wlan.fcs.bad_checksum (Frame Check Errors):")
        print("   Original values → Categories")

        def classify_fcs_errors(errors):
            if errors == 0:
                return 'No_Errors'
            elif errors <= 2:
                return 'Low_Error_Rate'
            elif errors <= 5:
                return 'Medium_Error_Rate'
            elif errors <= 10:
                return 'High_Error_Rate'
            else:
                return 'Critical_Error_Rate'

        df_binned['wlan.fcs.bad_checksum'] = df_binned['wlan.fcs.bad_checksum'].apply(classify_fcs_errors)

        print("   New categories distribution:")
        print(df_binned['wlan.fcs.bad_checksum'].value_counts())
        binning_summary.append('wlan.fcs.bad_checksum → error rate categories')

    # 6. Generic binning for remaining numerical features
    remaining_features = ['wlan.seq', 'radiotap.rxflags', 'wlan.rsn.capabilities.mfpc',
                         'radiotap.length', 'wlan.tag', 'wlan_radio.frequency', 'wlan_radio.phy']

    for feature in remaining_features:
        if feature in df_binned.columns and df_binned[feature].dtype in ['int64', 'float64']:
            print(f"\nBinning {feature} (Generic Quantile-Based):")

            # Skip if mostly zero values
            non_zero_pct = (df_binned[feature] != 0).sum() / len(df_binned)
            if non_zero_pct < 0.1:
                # Binary binning for mostly zero features
                def binary_classify(value):
                    return 'Present' if value != 0 else 'Absent'

                df_binned[feature] = df_binned[feature].apply(binary_classify)
                print(f"   Binary categories: {df_binned[feature].value_counts().to_dict()}")
                binning_summary.append(f'{feature} → binary categories (Present/Absent)')
            else:
                # Quartile-based binning for features with good data
                try:
                    quartiles = df_binned[feature].quantile([0.25, 0.5, 0.75]).tolist()

                    def quartile_classify(value):
                        if value <= quartiles[0]:
                            return 'Low_Range'
                        elif value <= quartiles[1]:
                            return 'Medium_Low_Range'
                        elif value <= quartiles[2]:
                            return 'Medium_High_Range'
                        else:
                            return 'High_Range'

                    df_binned[feature] = df_binned[feature].apply(quartile_classify)
                    print(f"   Quartile categories: {df_binned[feature].value_counts().to_dict()}")
                    binning_summary.append(f'{feature} → quartile categories')

                except Exception as e:
                    print(f"   Warning: Could not bin {feature}: {str(e)}")

    print(f"\nBINNING SUMMARY:")
    print(f"   Features binned: {len(binning_summary)}")
    for summary in binning_summary:
        print(f"{summary}")

    print(f"\nFINAL BINNED DATASET:")
    print(f"   Shape: {df_binned.shape} (same number of columns as input)")
    print(f"   All numerical features converted to categories")
    print(f"   Target columns preserved: {available_targets}")

    return df_binned

# Apply true binning
df_final = apply_true_binning(df_clean)

# =============================================================================
# STEP 6: FINAL DATASET SUMMARY
# =============================================================================
print("\n6. FINAL DATASET SUMMARY")
print("-" * 60)

def generate_final_summary(df_original, df_final, available_features, available_targets):
    """Generate comprehensive summary of the selection and binning process"""

    print("EXPERT FEATURE SELECTION AND TRUE BINNING COMPLETED")
    print("=" * 55)

    print(f"DATASET TRANSFORMATION:")
    print(f"   Original dataset: {df_original.shape}")
    print(f"   Selected columns: {len(available_features + available_targets)}")
    print(f"   Final dataset: {df_final.shape}")
    print(f"   Column count: UNCHANGED (true binning applied)")

    print(f"\nBINNED EXPERT FEATURES:")
    for i, feature in enumerate(available_features, 1):
        if feature in df_final.columns:
            unique_categories = df_final[feature].nunique()
            print(f"   {i:2d}. {feature} → {unique_categories} categories")

    print(f"\nTARGET COLUMNS (UNCHANGED):")
    for target in available_targets:
        if target in df_final.columns:
            unique_count = df_final[target].nunique()
            print(f"{target}: {unique_count} unique values")

    print(f"\n DATA TYPES AFTER BINNING:")
    dtype_counts = df_final.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"   {dtype}: {count} columns")

    print(f"\n BINNING BENEFITS:")
    print(f"    Categorical features ready for ML algorithms")
    print(f"    Reduced noise in numerical data")
    print(f"    Attack patterns captured in meaningful categories")
    print(f"    UAV-specific threats clearly identified")

    print(f"\n DATASET READY FOR:")
    print(f"    Machine Learning Model Training")
    print(f"    UAV Intrusion Detection Analysis")
    print(f"    Categorical Analysis and Visualization")
    print(f"    Attack Pattern Recognition")

# Generate final summary
generate_final_summary(df, df_final, available_features, available_targets)

# =============================================================================
# STEP 7: SAVE PROCESSED DATASET AS EXCEL
# =============================================================================
print(f"\ SAVING PROCESSED DATASET AS EXCEL")
print("-" * 60)

try:
    # Save the processed dataset as Excel
    output_filename = 'uav_expert_features_binned.xlsx'

    # Save with Excel writer for better formatting
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        df_final.to_excel(writer, sheet_name='UAV_Binned_Features', index=False)

        # Create a summary sheet
        summary_data = {
            'Metric': [
                'Original Dataset Shape',
                'Selected Features Count',
                'Target Columns Count',
                'Final Dataset Shape',
                'Binning Method',
                'File Created'
            ],
            'Value': [
                f"{df.shape[0]} rows x {df.shape[1]} columns",
                len(available_features),
                len(available_targets),
                f"{df_final.shape[0]} rows x {df_final.shape[1]} columns",
                'True Binning (Replace Values)',
                output_filename
            ]
        }

        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Processing_Summary', index=False)

    print(f" Processed dataset saved as: {output_filename}")
    print(f"   Sheet 1: 'UAV_Binned_Features' - Main binned dataset")
    print(f"   Sheet 2: 'Processing_Summary' - Processing information")
    print(f"   Shape: {df_final.shape}")
    print(f"   File size: {df_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

    # Display sample of binned data
    print(f"\n SAMPLE OF BINNED DATA:")
    print(df_final.head(3))

except Exception as e:
    print(f" Error saving Excel file: {str(e)}")
    print("Trying CSV backup...")
    try:
        df_final.to_csv('uav_expert_features_binned_backup.csv', index=False)
        print(f" Backup saved as CSV: uav_expert_features_binned_backup.csv")
    except Exception as csv_error:
        print(f" CSV backup also failed: {str(csv_error)}")

print(f"\n TRUE BINNING COMPLETED SUCCESSFULLY!")
print("="*80)

In [ ]:
# Encode Binned Categorical Features for Machine Learning
# Convert categorical binned values to numerical encoded values
# Author: Data Science Team

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("ENCODING BINNED CATEGORICAL FEATURES FOR MACHINE LEARNING")
print("="*80)

# =============================================================================
# STEP 1: LOAD BINNED DATASET
# =============================================================================
print("\n1. LOADING BINNED UAV-NIDD DATASET")
print("-" * 60)

def load_binned_dataset(file_path):
    """Load the binned dataset from Excel file"""
    try:
        print(f"Loading binned dataset from: {file_path}")

        # Load from Excel (main sheet)
        df = pd.read_excel(file_path, sheet_name='UAV_Binned_Features')

        print(" Binned dataset loaded successfully")
        print(f"   Dataset shape: {df.shape}")
        print(f"   Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

        # Check data types
        print(f"\ Data Types:")
        dtype_counts = df.dtypes.value_counts()
        for dtype, count in dtype_counts.items():
            print(f"   {dtype}: {count} columns")

        return df

    except Exception as e:
        print(f" Error loading binned dataset: {str(e)}")
        return None

# Load the binned dataset
BINNED_FILE_PATH = '/workspace/Crisp-dm/uav_expert_features_binned.xlsx'
df_binned = load_binned_dataset(BINNED_FILE_PATH)

if df_binned is None:
    print("Cannot proceed without binned dataset.")
    exit()

# =============================================================================
# STEP 2: ANALYZE CATEGORICAL FEATURES
# =============================================================================
print("\n2. ANALYZING CATEGORICAL FEATURES FOR ENCODING")
print("-" * 60)

def analyze_categorical_features(df):
    """Analyze categorical features before encoding"""

    print("CATEGORICAL FEATURE ANALYSIS:")
    print("-" * 40)

    # Separate features and targets
    target_columns = ['Label', 'category']
    feature_columns = [col for col in df.columns if col not in target_columns]

    print(f"Features to encode: {len(feature_columns)}")
    print(f"Target columns: {len(target_columns)}")

    # Analyze each feature
    encoding_info = {}

    for i, feature in enumerate(feature_columns, 1):
        unique_count = df[feature].nunique()
        sample_values = df[feature].value_counts().head(3)

        print(f"\n{i:2d}.  {feature}:")
        print(f"   Unique categories: {unique_count}")
        print(f"   Top 3 categories:")
        for category, count in sample_values.items():
            percentage = (count / len(df)) * 100
            print(f" {category}: {count:,} ({percentage:.1f}%)")

        # Determine encoding strategy
        if unique_count <= 10:
            strategy = "Label Encoding (≤10 categories)"
        elif unique_count <= 50:
            strategy = "Label Encoding (manageable size)"
        else:
            strategy = "Label Encoding (high cardinality - consider reduction)"

        print(f"Encoding strategy: {strategy}")

        encoding_info[feature] = {
            'unique_count': unique_count,
            'strategy': strategy,
            'sample_values': list(sample_values.index[:3])
        }

    # Analyze target columns
    print(f"\n TARGET COLUMNS ANALYSIS:")
    print("-" * 40)

    for target in target_columns:
        if target in df.columns:
            unique_count = df[target].nunique()
            print(f"\n{target}:")
            print(f"   Unique values: {unique_count}")

            value_counts = df[target].value_counts().head(5)
            for value, count in value_counts.items():
                percentage = (count / len(df)) * 100
                print(f"     • {value}: {count:,} ({percentage:.1f}%)")

            encoding_info[target] = {
                'unique_count': unique_count,
                'strategy': "Label Encoding (target)",
                'sample_values': list(value_counts.index[:3])
            }

    return feature_columns, target_columns, encoding_info

# Analyze categorical features
feature_columns, target_columns, encoding_info = analyze_categorical_features(df_binned)

# =============================================================================
# STEP 3: APPLY LABEL ENCODING TO ALL CATEGORICAL FEATURES
# =============================================================================
print("\n3. APPLYING LABEL ENCODING TO CATEGORICAL FEATURES")
print("-" * 60)

def encode_categorical_features(df, feature_columns, target_columns):
    """Apply label encoding to all categorical features"""

    df_encoded = df.copy()
    encoders = {}
    encoding_mappings = {}

    print("Encoding categorical features to numerical values...")

    # Encode feature columns
    print(f"\n ENCODING FEATURE COLUMNS:")
    print("-" * 40)

    for i, feature in enumerate(feature_columns, 1):
        print(f"\n{i:2d}. Encoding {feature}:")

        try:
            # Create label encoder
            encoder = LabelEncoder()

            # Fit and transform the feature
            df_encoded[feature] = encoder.fit_transform(df[feature].astype(str))

            # Store encoder for future use
            encoders[feature] = encoder

            # Create mapping for reference
            unique_categories = df[feature].unique()
            encoded_values = encoder.transform(unique_categories.astype(str))
            mapping = dict(zip(unique_categories, encoded_values))
            encoding_mappings[feature] = mapping

            # Display encoding information
            print(f"   Categories encoded: {len(unique_categories)}")
            print(f"   Encoding range: 0 to {max(encoded_values)}")

            # Show sample mappings (first 5)
            print(f"   Sample mappings:")
            for j, (original, encoded) in enumerate(list(mapping.items())[:5]):
                print(f"     '{original}' → {encoded}")

            if len(mapping) > 5:
                print(f"     ... and {len(mapping) - 5} more mappings")

        except Exception as e:
            print(f"    Error encoding {feature}: {str(e)}")
            continue

    # Encode target columns
    print(f"\n ENCODING TARGET COLUMNS:")
    print("-" * 40)

    for target in target_columns:
        if target in df.columns:
            print(f"\nEncoding {target}:")

            try:
                # Create label encoder for target
                encoder = LabelEncoder()

                # Fit and transform the target
                df_encoded[target] = encoder.fit_transform(df[target].astype(str))

                # Store encoder
                encoders[target] = encoder

                # Create mapping
                unique_values = df[target].unique()
                encoded_values = encoder.transform(unique_values.astype(str))
                mapping = dict(zip(unique_values, encoded_values))
                encoding_mappings[target] = mapping

                # Display encoding information
                print(f"   Classes encoded: {len(unique_values)}")
                print(f"   Encoding range: 0 to {max(encoded_values)}")

                print(f"   Class mappings:")
                for original, encoded in mapping.items():
                    count = (df[target] == original).sum()
                    percentage = (count / len(df)) * 100
                    print(f"     '{original}' → {encoded} ({count:,} samples, {percentage:.1f}%)")

            except Exception as e:
                print(f"    Error encoding {target}: {str(e)}")
                continue

    print(f"\n ENCODING SUMMARY:")
    print(f"   Features encoded: {len([f for f in feature_columns if f in encoders])}")
    print(f"   Targets encoded: {len([t for t in target_columns if t in encoders])}")
    print(f"   Total columns encoded: {len(encoders)}")

    return df_encoded, encoders, encoding_mappings

# Apply encoding
df_encoded, encoders, encoding_mappings = encode_categorical_features(
    df_binned, feature_columns, target_columns
)

# =============================================================================
# STEP 4: VALIDATE ENCODED DATASET
# =============================================================================
print("\n4. VALIDATING ENCODED DATASET")
print("-" * 60)

def validate_encoded_dataset(df_original, df_encoded, encoders):
    """Validate the encoded dataset"""

    print("VALIDATION RESULTS:")
    print("-" * 40)

    # Check shape consistency
    print(f" Shape consistency:")
    print(f"   Original: {df_original.shape}")
    print(f"   Encoded:  {df_encoded.shape}")
    print(f"   Match: {' Yes' if df_original.shape == df_encoded.shape else ' No'}")

    # Check data types
    print(f"\ Data type transformation:")
    original_dtypes = df_original.dtypes.value_counts()
    encoded_dtypes = df_encoded.dtypes.value_counts()

    print(f"   Original data types: {dict(original_dtypes)}")
    print(f"   Encoded data types:  {dict(encoded_dtypes)}")

    # Check for missing values
    print(f"\n Missing values check:")
    original_missing = df_original.isnull().sum().sum()
    encoded_missing = df_encoded.isnull().sum().sum()

    print(f"   Original missing: {original_missing}")
    print(f"   Encoded missing:  {encoded_missing}")
    print(f"   Status: {' Good' if encoded_missing == 0 else ' Warning'}")

    # Validate numeric ranges
    print(f"\ Numeric range validation:")
    for column in df_encoded.columns:
        if df_encoded[column].dtype in ['int64', 'int32']:
            min_val = df_encoded[column].min()
            max_val = df_encoded[column].max()
            print(f"   {column}: [{min_val}, {max_val}]")

    # Sample comparison
    print(f"\n Sample data comparison:")
    print("Original (first 3 rows, first 5 columns):")
    print(df_original.iloc[:3, :5])
    print("\nEncoded (first 3 rows, first 5 columns):")
    print(df_encoded.iloc[:3, :5])

    return True

# Validate encoded dataset
validation_result = validate_encoded_dataset(df_binned, df_encoded, encoders)

# =============================================================================
# STEP 5: CREATE ENCODING REFERENCE DOCUMENTATION
# =============================================================================
print("\n5. CREATING ENCODING REFERENCE DOCUMENTATION")
print("-" * 60)

def create_encoding_reference(encoding_mappings, encoders):
    """Create comprehensive encoding reference documentation"""

    print("Creating encoding reference documentation...")

    # Create detailed mapping dataframes for each feature
    reference_dfs = {}

    for feature, mapping in encoding_mappings.items():
        # Create dataframe for this feature's mapping
        mapping_data = []
        for original, encoded in mapping.items():
            mapping_data.append({
                'Feature': feature,
                'Original_Value': original,
                'Encoded_Value': encoded,
                'Data_Type': 'Target' if feature in target_columns else 'Feature'
            })

        reference_df = pd.DataFrame(mapping_data)
        reference_dfs[feature] = reference_df

    # Combine all mappings into one reference dataframe
    all_mappings = []
    for feature_df in reference_dfs.values():
        all_mappings.append(feature_df)

    complete_reference = pd.concat(all_mappings, ignore_index=True)

    print(f" Encoding reference created:")
    print(f"   Total mappings: {len(complete_reference)}")
    print(f"   Features documented: {len(reference_dfs)}")

    # Display sample of reference
    print(f"\ SAMPLE ENCODING REFERENCE:")
    print(complete_reference.head(10))

    return complete_reference, reference_dfs

# Create encoding reference
encoding_reference, feature_references = create_encoding_reference(encoding_mappings, encoders)

# =============================================================================
# STEP 6: SAVE ENCODED DATASET AND REFERENCES
# =============================================================================
print("\n6. SAVING ENCODED DATASET AND REFERENCES")
print("-" * 60)

def save_encoded_results(df_encoded, encoding_reference, encoders, encoding_mappings):
    """Save encoded dataset and all reference materials"""

    try:
        output_filename = 'uav_encoded_features.xlsx'

        print(f"Saving to: {output_filename}")

        # Save with multiple sheets
        with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:

            # Main encoded dataset
            df_encoded.to_excel(writer, sheet_name='Encoded_Dataset', index=False)
            print(" Sheet 1: 'Encoded_Dataset' - Main numerical dataset")

            # Complete encoding reference
            encoding_reference.to_excel(writer, sheet_name='Encoding_Reference', index=False)
            print(" Sheet 2: 'Encoding_Reference' - All mappings")

            # Summary statistics
            summary_data = {
                'Metric': [
                    'Total Samples',
                    'Total Features',
                    'Target Columns',
                    'Encoded Features',
                    'Encoded Targets',
                    'Total Unique Mappings',
                    'File Created',
                    'Ready for ML'
                ],
                'Value': [
                    f"{df_encoded.shape[0]:,}",
                    len(feature_columns),
                    len(target_columns),
                    len([f for f in feature_columns if f in encoders]),
                    len([t for t in target_columns if t in encoders]),
                    len(encoding_reference),
                    output_filename,
                    'Yes'
                ]
            }

            summary_df = pd.DataFrame(summary_data)
            summary_df.to_excel(writer, sheet_name='Summary', index=False)
            print("Sheet 3: 'Summary' - Processing summary")

            # Feature-wise encoding details
            for i, (feature, ref_df) in enumerate(feature_references.items()):
                if i < 20:  # Limit to first 20 features to avoid too many sheets
                    sheet_name = f"{feature[:25]}_mapping"  # Limit sheet name length
                    ref_df.to_excel(writer, sheet_name=sheet_name, index=False)

            if len(feature_references) > 20:
                print(f"Individual mapping sheets created for first 20 features")
            else:
                print(f" Individual mapping sheets created for all {len(feature_references)} features")

        print(f"\n SAVED FILES:")
        print(f"    {output_filename}")
        print(f"    Shape: {df_encoded.shape}")
        print(f"    Size: {df_encoded.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

        # Display final sample
        print(f"\n FINAL ENCODED SAMPLE:")
        print("First 3 rows of encoded dataset:")
        print(df_encoded.head(3))

        return True

    except Exception as e:
        print(f" Error saving encoded results: {str(e)}")
        return False

# Save encoded results
save_success = save_encoded_results(df_encoded, encoding_reference, encoders, encoding_mappings)

# =============================================================================
# STEP 7: FINAL SUMMARY AND NEXT STEPS
# =============================================================================
print("\n7. FINAL SUMMARY - ENCODING COMPLETED")
print("-" * 60)

def generate_final_encoding_summary(df_original, df_encoded, encoders, encoding_mappings):
    """Generate comprehensive summary of encoding process"""

    print("CATEGORICAL ENCODING COMPLETED SUCCESSFULLY")
    print("=" * 50)

    print(f" TRANSFORMATION OVERVIEW:")
    print(f"   Original dataset: {df_original.shape}")
    print(f"   Encoded dataset:  {df_encoded.shape}")
    print(f"   Data preservation: Complete")

    print(f"\n ENCODING STATISTICS:")
    print(f"   Features encoded: {len([f for f in feature_columns if f in encoders])}/{len(feature_columns)}")
    print(f"   Targets encoded:  {len([t for t in target_columns if t in encoders])}/{len(target_columns)}")
    print(f"   Total mappings:   {sum(len(mapping) for mapping in encoding_mappings.values())}")

    print(f"\n DATA TYPE TRANSFORMATION:")
    print(f"   Before: All categorical (object)")
    print(f"   After:  All numerical (int64)")

    print(f"\ KEY FEATURES ENCODED:")
    important_features = ['frame.len', 'radiotap.channel.flags.cck', 'udp.srcport', 'Label']
    for feature in important_features:
        if feature in encoding_mappings:
            categories = len(encoding_mappings[feature])
            print(f"   • {feature}: {categories} categories → 0-{categories-1}")



# Generate final summary
generate_final_encoding_summary(df_binned, df_encoded, encoders, encoding_mappings)

print(f"\n CATEGORICAL ENCODING COMPLETED SUCCESSFULLY!")
print("="*80)

**APPROACH TWO & APPROACH THREE : Data Augmentation with SMOTE and BORDERLINE-SMOTE**

In [ ]:
# ==============================================================================
# DATA PREPARATION 
# ==============================================================================


# ------------------------------------------------------------------------------
# 0. SETUP AND INSTALLATION
# ------------------------------------------------------------------------------
!pip install -q imbalanced-learn
import os # <<< THE FIX IS HERE: Import the 'os' module
import pandas as pd
import numpy as np

# ------------------------------------------------------------------------------
# 1. CONNECT TO DATA SOURCE AND LOAD
# ------------------------------------------------------------------------------

# UPDATED: Using the pre-encoded Excel file
file_path = "/workspace/Crisp-dm/uav_encoded_features.xlsx"

# Define the output directory and file path for the cleaned data
output_dir_dataprep = "/workspace/Crisp-dm/Data_Preparation"
os.makedirs(output_dir_dataprep, exist_ok=True)
cleaned_excel_path = os.path.join(output_dir_dataprep, "final_cleaned_encoded_dataset.xlsx")

try:
    # Assuming the data is on the first sheet, so no sheet_name needed.
    df = pd.read_excel(file_path)
    print(f"\nSuccessfully loaded encoded data from: {file_path}")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: The file was not found at the specified path: {file_path}")
    exit()
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    exit()

# ------------------------------------------------------------------------------
# 2. VALIDATE DATA AND EXCLUDE PROBLEMATIC CLASS
# ------------------------------------------------------------------------------
# Using 'Label' as the target column. Standardize if necessary.
if 'Label' not in df.columns and 'label' in df.columns:
    df.rename(columns={'label': 'Label'}, inplace=True) # Standardize to 'Label'

if 'Label' not in df.columns:
    print("Error: A target column named 'Label' or 'label' was not found.")
    exit()

print("\nOriginal Class distribution in the dataset:")
print(df['Label'].value_counts())

# *** THE CRITICAL FIX IS HERE ***
# We must exclude the class with too few samples for SMOTE and CV to work.
# We know from previous analysis that the class with label '9' is the problem.
problematic_class_label = 9
original_rows = len(df)

if problematic_class_label in df['Label'].unique():
    df = df[df['Label'] != problematic_class_label]
    print(f"\nACTION: Excluded class '{problematic_class_label}'. Removed {original_rows - len(df)} rows.")
    print("\nNew Class distribution:")
    print(df['Label'].value_counts())
else:
    print(f"\nINFO: Class '{problematic_class_label}' not found. No exclusion needed.")


# ------------------------------------------------------------------------------
# 3. SAVE THE FINAL CLEANED DATASET
# ------------------------------------------------------------------------------
try:
    df.to_excel(cleaned_excel_path, index=False)
    print(f"\nSUCCESS: Final cleaned dataset saved to: {cleaned_excel_path}")
except Exception as e:
    print(f"\nERROR: Could not save the cleaned file. Error: {e}")

# ------------------------------------------------------------------------------
# 4. FINAL PREPARATION FOR MODELING
# ------------------------------------------------------------------------------
# All columns except 'Label' are features
feature_columns = [col for col in df.columns if col != 'Label']
target_column = 'Label'

X = df[feature_columns]
y = df[target_column]

# Final verification that all data is numeric
if not all(X.dtypes.apply(pd.api.types.is_numeric_dtype)):
    print("\nWarning: Non-numeric data found in features. Please check the Excel file.")
else:
    print(f"\nData preparation complete. All {X.shape[1]} features are numeric.")
    print(f"The variables 'X' and 'y' for the {len(y.unique())} main classes are now ready for the modeling cells.")

**VISUALIZATION OF THE RESAMPLING EFFECT**

In [ ]:
# ==============================================================================
# CELL 1.5: VISUALIZATION OF THE RESAMPLING EFFECT
# ==============================================================================
# This cell demonstrates the effect of our combined sampling strategy on the dataset.
# It fits the pipeline ONCE to the full dataset just for visualization purposes.

# Required libraries for this cell
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

print("Demonstrating the effect of the resampling pipeline...")


label_mapping = {
    1: 'DDoS',
    0: 'BruteForce',
    11: 'UDP Flooding',
    7: 'MITM',
    5: 'ICMP Flooding',
    6: 'Jamming',
    12: 'replay',
    4: 'FakeLanding',
    8: 'Normal',
    2: 'De-authentication',
    10: 'Scanning',
    3: 'DoS',
    9: 'Reconnassiance'
}

# Define the Undersampling and Oversampling steps exactly as in the modeling cell
initial_under_sampler = RandomUnderSampler(
    sampling_strategy={1: 50000, 0: 50000, 11: 50000, 7: 50000},
    random_state=42
)
over_sampler = SMOTE(random_state=42, k_neighbors=3)

# Create the demonstration pipeline (no scaler or model needed for this)
vis_pipeline = Pipeline([
    ('initial_undersampling', initial_under_sampler),
    ('oversampling_smote', over_sampler)
])

# --- Apply the pipeline to the full dataset to get an example of resampled data ---
X_resampled, y_resampled = vis_pipeline.fit_resample(X, y)

print(f"\nOriginal dataset size: {len(y)}")
print(f"Resampled dataset size: {len(y_resampled)}")

# --- Create the visualizations ---
# Convert the numeric resampled labels to their real names for plotting
y_resampled_named = y_resampled.map(label_mapping)

# Calculate class distribution
class_distribution = y_resampled_named.value_counts()

# Create the bar plot
plt.figure(figsize=(12, 8))
sns.barplot(x=class_distribution.index, y=class_distribution.values, palette='viridis')
plt.title('Class Distribution After Combined Over- and Under-Sampling', fontsize=16)
plt.xlabel('Attack Type', fontsize=12)
plt.ylabel('Number of Samples', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# --- Display the exact counts and percentages ---
distribution_df = pd.DataFrame({
    'Count': class_distribution,
    'Percentage': (class_distribution / len(y_resampled) * 100).round(2)
})
print("\nDistribution of classes in the resampled dataset:")
print(distribution_df)

# MODELING

**APPROACH ONE : Original Imbalanced Dataset**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os  # Added import for os module

from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC # Importing both types of SVM
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# ==============================================================================
# 1. FULL DATA LOADING
# ==============================================================================

file_path = "/workspace/Crisp-dm/uav_encoded_features.xlsx"

# Define the output directory for this phase's results
output_dir_modeling = "/workspace/Crisp-dm/Modeling"
os.makedirs(output_dir_modeling, exist_ok=True)
original_results_path = os.path.join(output_dir_modeling, "original_imbalanced_baseline_results.xlsx")

try:
    df = pd.read_excel(file_path)
    print(f"Full dataset loaded: {df.shape[0]} rows.")
except FileNotFoundError:
    print(f"Error: File not found at the address: {file_path}")
    exit()

print("\nClass distribution:")
print(df['Label'].value_counts())

feature_columns = [
    'frame.len', 'radiotap.channel.flags.cck', 'wlan.seq', 'radiotap.dbm_antsignal',
    'radiotap.rxflags', 'wlan.rsn.capabilities.mfpc', 'radiotap.length',
    'wlan.fcs.bad_checksum', 'wlan.tag', 'wlan_radio.frequency',
    'wlan_radio.phy', 'udp.srcport'
]
target_column = 'Label'

# Validate columns
missing_columns = [col for col in feature_columns if col not in df.columns]
if missing_columns:
    print(f"Error: Missing columns: {missing_columns}")
    exit()

X = df[feature_columns]
y = df[target_column]

# ==============================================================================
# 2. DEFINITION OF OPTIMIZED MODELS
# ==============================================================================

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced'),
    "k-NN": KNeighborsClassifier(), # Can also be slow, but less than SVC
    "Decision Tree": DecisionTreeClassifier(random_state=42, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1), # n_jobs=-1 to use all cores
    "XGBoost": XGBClassifier(eval_metric='mlogloss', random_state=42, n_jobs=-1),

    # --- STRATEGY 1 (RECOMMENDED): FAST LINEAR SVM ---
    "Linear SVM": LinearSVC(random_state=42, class_weight='balanced', dual=False, max_iter=3000),

    # --- STRATEGY 2 (OPTIONAL, VERY SLOW): RBF SVM WITH INCREASED CACHE ---
    # Uncomment the following line ONLY if you want to test and have time and RAM.
    # "SVM (RBF)": SVC(kernel='rbf', random_state=42, class_weight='balanced', cache_size=2000), # Cache increased to 2000MB

    "Neural Network (MLP)": MLPClassifier(max_iter=1000, random_state=42, early_stopping=True)
}

CV_FOLDS = 5
kfold = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=42)
cv_results = {}

# ==============================================================================
# 3. EXECUTION OF CROSS-VALIDATION
# ==============================================================================

print("\nStarting cross-validation on the entire dataset...")
total_start_time = time.time()

for name, model in models.items():
    print(f"\n--- Evaluating model: {name} ---")
    start_time = time.time()

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])

    scoring_metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
    scores = cross_validate(pipeline, X, y, cv=kfold, scoring=scoring_metrics, n_jobs=-1) # n_jobs=-1 to parallelize

    end_time = time.time()
    elapsed_time = end_time - start_time
    cv_results[name] = scores
    print(f"Evaluation finished for {name} in {elapsed_time:.2f} seconds.")

total_end_time = time.time()
print(f"\nComplete evaluation finished in {(total_end_time - total_start_time)/60:.2f} minutes.")


# ==============================================================================
# 4. PRESENTATION AND COMPARISON OF RESULTS
# ==============================================================================
print("\n" + "="*50)
print("PERFORMANCE REPORT (CROSS-VALIDATION)")
print("="*50)

results_summary = []
for name, scores in cv_results.items():
    results_summary.append({
        'Model': name,
        'Accuracy (Mean)': scores['test_accuracy'].mean(),
        'F1-Score (Mean)': scores['test_f1_macro'].mean(),
        'Precision': scores['test_precision_macro'].mean(),
        'Recall (Mean)': scores['test_recall_macro'].mean()
    })

results_df = pd.DataFrame(results_summary)
results_df = results_df.sort_values(by='F1-Score (Mean)', ascending=False)
print(results_df.to_string(index=False, float_format="%.4f"))

# ------------------------------------------------------------------------------
# 5. SAVE THE RESULTS TO AN EXCEL FILE
# ------------------------------------------------------------------------------
try:
    results_df.to_excel(original_results_path, index=False)
    print(f"\nSUCCESS: Results for original imbalanced data saved to:")
    print(original_results_path)
except Exception as e:
    print(f"\nERROR: Could not save the results file. Error: {e}")

**APPROACH TWO : Data Augmentation with SMOTE**

In [ ]:
# ===============================================================================
# CELL 2: MODELING - SMOTE BASELINE 
# ===============================================================================
# This cell loads the cleaned data and establishes the SMOTE baseline using the
# more robust "Scale Before Resample" strategy for a more realistic evaluation.
# It saves the final, comprehensive report to a file.

# ------------------------------------------------------------------------------
# 0. REQUIRED LIBRARIES FOR THIS CELL
# ------------------------------------------------------------------------------
import time
import pandas as pd
import numpy as np
import os
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Import all 7 models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# ------------------------------------------------------------------------------
# 1. LOAD CLEANED DATA AND DEFINE PATHS
# ------------------------------------------------------------------------------
data_prep_dir = "/workspace/Crisp-dm/Data_Preparation"
cleaned_data_path = os.path.join(data_prep_dir, "final_cleaned_encoded_dataset.xlsx")
output_dir_modeling = "/workspace/Crisp-dm/Modeling"
os.makedirs(output_dir_modeling, exist_ok=True)
smote_results_path = os.path.join(output_dir_modeling, "smote_realistic_baseline_results.xlsx")

try:
    df = pd.read_excel(cleaned_data_path)
    X = df.drop('Label', axis=1)
    y = df['Label']
    print(f"Successfully loaded cleaned dataset. Shape: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: Cleaned data file not found at '{cleaned_data_path}'. Please run Cell 1 first.")
    exit()

# ------------------------------------------------------------------------------
# 2. MANUAL CROSS-VALIDATION LOOP (WITH REALISTIC SCALING)
# ------------------------------------------------------------------------------
print("\nStarting SMOTE Baseline Evaluation (with Realistic Scaling)...")
print("-" * 60)
model_names = [ "Logistic Regression", "k-NN", "Decision Tree", "Random Forest", "Linear SVM", "XGBoost", "Neural Network (MLP)" ]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
smote_results_storage = {name: [] for name in model_names}

for fold_num, (train_index, test_index) in enumerate(kfold.split(X, y), 1):
    print(f"\n===== Processing Fold {fold_num}/5 =====")
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

    # --- THE NEW, MORE REALISTIC ORDER ---

    # 1. SCALING FIRST: Fit scaler ONLY on the original training data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test) # Apply the same scaling to the test set

    # Convert scaled training data back to a DataFrame for imblearn compatibility
    X_train_scaled_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)

    # 2. RESAMPLING SECOND: Apply to the now-scaled training data
    class_counts = Counter(y_train)
    under_sampling_dict = {label: min(count, 50000) for label, count in class_counts.items() if count > 20000}
    initial_under_sampler = RandomUnderSampler(sampling_strategy=under_sampling_dict, random_state=42)
    X_train_under, y_train_under = initial_under_sampler.fit_resample(X_train_scaled_df, y_train)

    min_class_count = min(Counter(y_train_under).values())
    smote_k = min(min_class_count - 1, 5)
    over_sampler = SMOTE(random_state=42, k_neighbors=max(1, smote_k))
    X_resampled, y_resampled = over_sampler.fit_resample(X_train_under, y_train_under)

    # 3. LABEL ENCODING (remains the same)
    le = LabelEncoder().fit(pd.concat([y_resampled, y_test]))
    y_resampled_encoded = le.transform(y_resampled)
    y_test_encoded = le.transform(y_test)

    models_this_fold = {
        "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42), "k-NN": KNeighborsClassifier(),
        "Decision Tree": DecisionTreeClassifier(random_state=42), "Random Forest": RandomForestClassifier(random_state=42, n_jobs=-1),
        "Linear SVM": LinearSVC(random_state=42, dual=False, max_iter=3000),
        "XGBoost": XGBClassifier(random_state=42, eval_metric='mlogloss', use_label_encoder=False),
        "Neural Network (MLP)": MLPClassifier(random_state=42, early_stopping=True)
    }

    for name, model in models_this_fold.items():
        start_time = time.time()
        # Train on the resampled data (which is already scaled)
        model.fit(X_resampled, y_resampled_encoded)
        # Test on the scaled test data
        y_pred_encoded = model.predict(X_test_scaled)

        scores = { 'accuracy': accuracy_score(y_test_encoded, y_pred_encoded), 'precision': precision_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0), 'recall': recall_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0), 'f1_score': f1_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0) }
        smote_results_storage[name].append(scores)
        end_time = time.time()
        print(f"  - {name} trained and evaluated in {end_time - start_time:.2f}s")

# ------------------------------------------------------------------------------
# 3. AGGREGATE, DISPLAY, AND SAVE REPORT
# ------------------------------------------------------------------------------
report_data_smote = []
for name, fold_scores in final_results.items():
    avg_scores = {
        'Model': name,
        'Accuracy': np.mean([s['accuracy'] for s in fold_scores]),
        'Precision': np.mean([s['precision'] for s in fold_scores]),
        'Recall': np.mean([s['recall'] for s in fold_scores]),
        'F1-Score': np.mean([s['f1_score'] for s in fold_scores]),
    }

    # --- **THE KEY CHANGE IS HERE: Get ALL parameters** ---
    # Get the model instance from the last fold
    model_instance = models_this_fold[name]
    model_params = model_instance.get_params(deep=False) # deep=False gives cleaner output

    # Add all parameters to the results dictionary
    avg_scores.update(model_params)
    report_data.append(avg_scores)


results_df_smote = pd.DataFrame(report_data_smote).sort_values(by='F1-Score', ascending=False)
print("\n\n" + "="*70)
print("FINAL COMPREHENSIVE REPORT (SMOTE WITH REALISTIC SCALING)")
print("="*70)
print(results_df_smote.to_string(index=False, float_format="%.4f"))

try:
    results_df_smote.to_excel(smote_results_path, index=False)
    print(f"\nSUCCESS: SMOTE baseline report saved to: {smote_results_path}")
except Exception as e:
    print(f"\nERROR: Could not save results file. Error: {e}")

**APPROACH THREE : Data Augmentation with BORDERLINE-SMOTE**

In [ ]:
# ==============================================================================
# CELL 3: MODELING - BORDERLINE-SMOTE
# ==============================================================================
# This cell loads the cleaned data and establishes the Borderline-SMOTE baseline
# using the robust "Scale Before Resample" strategy for a more realistic evaluation.
# It saves the final, comprehensive report to a file.

# ------------------------------------------------------------------------------
# 0. REQUIRED LIBRARIES FOR THIS CELL
# ------------------------------------------------------------------------------
import time
import pandas as pd
import numpy as np
import os
from collections import Counter
from imblearn.over_sampling import BorderlineSMOTE # <<< THE KEY CHANGE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Import all 7 models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# ------------------------------------------------------------------------------
# 1. LOAD CLEANED DATA AND DEFINE PATHS
# ------------------------------------------------------------------------------
data_prep_dir = "/workspace/Crisp-dm/Data_Preparation"
cleaned_data_path = os.path.join(data_prep_dir, "final_cleaned_encoded_dataset.xlsx")
output_dir_modeling = "/workspace/Crisp-dm/Modeling"
os.makedirs(output_dir_modeling, exist_ok=True)
borderline_results_path = os.path.join(output_dir_modeling, "borderline_smote_realistic_baseline_results.xlsx")

try:
    df = pd.read_excel(cleaned_data_path)
    X = df.drop('Label', axis=1)
    y = df['Label']
    print(f"Successfully loaded cleaned dataset. Shape: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: Cleaned data file not found at '{cleaned_data_path}'. Please run Cell 1 first.")
    exit()

# ------------------------------------------------------------------------------
# 2. MANUAL CROSS-VALIDATION LOOP (WITH REALISTIC SCALING)
# ------------------------------------------------------------------------------
print("\nStarting Borderline-SMOTE Baseline Evaluation (with Realistic Scaling)...")
print("-" * 60)
model_names = [ "Logistic Regression", "k-NN", "Decision Tree", "Random Forest", "Linear SVM", "XGBoost", "Neural Network (MLP)" ]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
borderline_results_storage = {name: [] for name in model_names}

for fold_num, (train_index, test_index) in enumerate(kfold.split(X, y), 1):
    print(f"\n===== Processing Fold {fold_num}/5 with Borderline-SMOTE =====")
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

    # --- THE NEW, MORE REALISTIC ORDER ---

    # 1. SCALING FIRST
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_train_scaled_df = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)

    # 2. RESAMPLING SECOND
    class_counts = Counter(y_train)
    under_sampling_dict = {label: min(count, 50000) for label, count in class_counts.items() if count > 20000}
    initial_under_sampler = RandomUnderSampler(sampling_strategy=under_sampling_dict, random_state=42)
    X_train_under, y_train_under = initial_under_sampler.fit_resample(X_train_scaled_df, y_train)

    min_class_count = min(Counter(y_train_under).values())
    bsmote_k = min(min_class_count - 1, 5)

    # *** Using BorderlineSMOTE ***
    over_sampler = BorderlineSMOTE(random_state=42, k_neighbors=max(1, bsmote_k))
    X_resampled, y_resampled = over_sampler.fit_resample(X_train_under, y_train_under)

    # 3. LABEL ENCODING
    le = LabelEncoder().fit(pd.concat([y_resampled, y_test]))
    y_resampled_encoded = le.transform(y_resampled)
    y_test_encoded = le.transform(y_test)

    models_this_fold = {
        "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42), "k-NN": KNeighborsClassifier(),
        "Decision Tree": DecisionTreeClassifier(random_state=42), "Random Forest": RandomForestClassifier(random_state=42, n_jobs=-1),
        "Linear SVM": LinearSVC(random_state=42, dual=False, max_iter=3000),
        "XGBoost": XGBClassifier(random_state=42, eval_metric='mlogloss', use_label_encoder=False),
        "Neural Network (MLP)": MLPClassifier(random_state=42, early_stopping=True)
    }

    for name, model in models_this_fold.items():
        start_time = time.time()
        model.fit(X_resampled, y_resampled_encoded)
        y_pred_encoded = model.predict(X_test_scaled)

        scores = { 'accuracy': accuracy_score(y_test_encoded, y_pred_encoded), 'precision': precision_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0), 'recall': recall_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0), 'f1_score': f1_score(y_test_encoded, y_pred_encoded, average='macro', zero_division=0) }
        borderline_results_storage[name].append(scores)
        end_time = time.time()
        print(f"  - {name} trained and evaluated in {end_time - start_time:.2f}s")

# ------------------------------------------------------------------------------
# 3. AGGREGATE, DISPLAY, AND SAVE REPORT
# ------------------------------------------------------------------------------
report_data_borderline = []
for name, fold_scores in borderline_results_storage.items():
    avg_scores = {
        'Model': name,
        'Accuracy': np.mean([s['accuracy'] for s in fold_scores]),
        'Precision': np.mean([s['precision'] for s in fold_scores]),
        'Recall': np.mean([s['recall'] for s in fold_scores]),
        'F1-Score': np.mean([s['f1_score'] for s in fold_scores]),
    }

    # --- **THE KEY CHANGE IS HERE: Get ALL parameters** ---
    # Get the model instance from the last fold
    model_instance = models_this_fold[name]
    model_params = model_instance.get_params(deep=False)

    # Add all parameters to the results dictionary
    avg_scores.update(model_params)
    report_data_borderline.append(avg_scores)

results_df_borderline = pd.DataFrame(report_data_borderline).sort_values(by='F1-Score', ascending=False)
print("\n\n" + "="*70)
print("FINAL COMPREHENSIVE REPORT (BORDERLINE-SMOTE WITH REALISTIC SCALING)")
print("="*70)
print(results_df_borderline.to_string(index=False, float_format="%.4f"))

try:
    results_df_borderline.to_excel(borderline_results_path, index=False)
    print(f"\nSUCCESS: Borderline-SMOTE baseline report saved to: {borderline_results_path}")
except Exception as e:
    print(f"\nERROR: Could not save results file. Error: {e}")

# EVALUATION

**APPROACH ONE : Original Imbalanced Dataset**

In [ ]:
import pandas as pd
import numpy as np
import time
import os
from scipy.stats import randint

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Import the models
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# ==============================================================================
# 1. DEFINE PATHS AND LOAD DATA
# ==============================================================================

file_path = "/workspace/Crisp-dm/uav_encoded_features.xlsx"
modeling_dir = "/workspace/Crisp-dm/Modeling"
evaluation_dir = "/workspace/Crisp-dm/Evaluation"
os.makedirs(evaluation_dir, exist_ok=True)

# Path to the previously saved UNTUNED baseline results
original_baseline_path = os.path.join(modeling_dir, "original_imbalanced_baseline_results.xlsx")
# Path for the NEW TUNED results
tuning_results_path = os.path.join(evaluation_dir, "original_imbalanced_tuning_results_complete.xlsx")

try:
    df = pd.read_excel(file_path)
    print(f"Full dataset loaded: {df.shape[0]} rows.")
except FileNotFoundError:
    print(f"Error: File not found at: {file_path}")
    exit()

if 'Label' not in df.columns and 'label' in df.columns:
    df.rename(columns={'label': 'Label'}, inplace=True)

X = df.drop('Label', axis=1)
y = df['Label']

# ==============================================================================
# 2. DEFINE PARAMETER GRIDS FOR TUNING
# ==============================================================================

param_grids = {
    "Random Forest": { 'model__n_estimators': randint(100, 400), 'model__max_depth': randint(10, 50), 'model__min_samples_split': [2, 5, 10], 'model__min_samples_leaf': [1, 2, 4], 'model__class_weight': ['balanced'] },
    "Decision Tree": { 'model__max_depth': randint(5, 40), 'model__min_samples_split': [2, 10, 20], 'model__min_samples_leaf': [1, 5, 10], 'model__criterion': ['gini', 'entropy'], 'model__class_weight': ['balanced'] },
    "Neural Network (MLP)": { 'model__hidden_layer_sizes': [(50, 50), (100,), (100, 50)], 'model__alpha': [0.0001, 0.001, 0.01], 'model__learning_rate_init': [0.001, 0.01], 'model__activation': ['relu', 'tanh'] }
}

# We are only tuning these three models now
models_to_tune = {
    "Random Forest": RandomForestClassifier(random_state=42, n_jobs=-1),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Neural Network (MLP)": MLPClassifier(random_state=42, early_stopping=True)
}

# ==============================================================================
# 3. EXECUTE THE RANDOMIZED SEARCH (FOR STABLE MODELS)
# ==============================================================================
CV_FOLDS = 3
kfold = StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=42)
N_ITER_SEARCH = 15
tuning_results = [] # This will hold the new results

print(f"Starting hyperparameter tuning for {len(models_to_tune)} models...")
print("-" * 50)

for name, model in models_to_tune.items():
    print(f"\n>>> Tuning model: {name}")
    start_time = time.time()
    pipeline = Pipeline([ ('scaler', StandardScaler()), ('model', model) ])
    random_search = RandomizedSearchCV( estimator=pipeline, param_distributions=param_grids[name], n_iter=N_ITER_SEARCH, cv=kfold, scoring='f1_macro', n_jobs=-1, random_state=42, verbose=1 )
    random_search.fit(X, y)

    # Store all results in a structured way
    result_data = {
        'Model': name,
        'Best F1-Score (Tuned)': random_search.best_score_,
        'Status': 'Tuned' # Mark as tuned
    }
    for param, value in random_search.best_params_.items():
        result_data[param.replace('model__', '')] = value
    tuning_results.append(result_data)

    end_time = time.time()
    print(f"Finished tuning {name} in {(end_time - start_time) / 60:.2f} minutes.")
    print(f"Best F1-Score found: {random_search.best_score_:.4f}")
    print("-" * 50)

# ==============================================================================
# 4. FINAL SUMMARY AND SAVING OF TUNING RESULTS
# ==============================================================================
print("\n\n" + "="*60)
print("HYPERPARAMETER TUNING FINAL REPORT")
print("="*60)

# Create a DataFrame from the new tuning results
new_results_df = pd.DataFrame(tuning_results)

# --- **THE FIX IS HERE: Load old results and merge** ---
try:
    print("Loading previous baseline results to include XGBoost...")
    baseline_df = pd.read_excel(original_baseline_path)

    # Find the row for XGBoost in the old results
    xgboost_baseline = baseline_df[baseline_df['Model'] == 'XGBoost'].copy()

    if not xgboost_baseline.empty:
        # Rename columns to match the new format
        xgboost_baseline.rename(columns={'F1-Score (Mean)': 'Best F1-Score (Tuned)'}, inplace=True)
        xgboost_baseline['Status'] = 'Default (Baseline)' # Mark as untuned

        # Combine the new tuned results with the old XGBoost baseline
        final_df = pd.concat([new_results_df, xgboost_baseline], ignore_index=True)
        print("Successfully merged XGBoost baseline results.")
    else:
        print("Warning: XGBoost not found in the baseline file. Using only new results.")
        final_df = new_results_df

except FileNotFoundError:
    print(f"Warning: Baseline results file not found at '{original_baseline_path}'.")
    print("Displaying only the newly tuned results.")
    final_df = new_results_df

# Sort the final combined DataFrame
final_df = final_df.sort_values(by='Best F1-Score (Tuned)', ascending=False)

# Display the final results table
print("\n--- Combined Performance Report ---")
print(final_df.to_string(index=False, float_format="%.4f"))

# Save the comprehensive results to the specified Excel file
try:
    final_df.to_excel(tuning_results_path, index=False)
    print(f"\n\nSUCCESS: Full combined tuning results saved to:")
    print(tuning_results_path)
except Exception as e:
    print(f"\n\nERROR: Could not save the tuning results file. Error: {e}")

**APPROACH TWO : Data Augmentation with SMOTE**

In [ ]:
# ==============================================================================
# Evaluation: SMOTE BASELINE Fine-Tuning
# ==============================================================================
# This version accepts that XGBoost may fail tuning on a sample and corrects

# Required libraries
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_validate
from scipy.stats import randint
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImblearnPipeline # Use a different name to avoid confusion
from sklearn.pipeline import Pipeline as SklearnPipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# This cell assumes Cell 1 (Data Prep) has been run and 'X' and 'y' exist.

# ------------------------------------------------------------------------------
# PART 1: FAST HYPERPARAMETER TUNING ON A DATA SAMPLE
# ------------------------------------------------------------------------------
print("="*60)
print("EVALUATION PHASE, PART 1: Finding Best Hyperparameters on a Data Sample")
print("="*60)

# Define path to the CLEANED data file created in the Data Preparation phase
data_prep_dir = "/workspace/Crisp-dm/Data_Preparation"
cleaned_data_path = os.path.join(data_prep_dir, "final_cleaned_encoded_dataset.xlsx")

try:
    df = pd.read_excel(cleaned_data_path)
    print(f"Successfully loaded cleaned dataset from: {cleaned_data_path}")
    print(f"Dataset shape: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: Cleaned data file not found at '{cleaned_data_path}'.")
    print("Please ensure Cell 1 (Data Preparation) has been run successfully to create this file.")
    exit()

# Create X and y from the loaded data
X = df.drop('Label', axis=1)
y = df['Label']


# Define the output directory and file path for the final report
output_dir_evaluation = "/workspace/Crisp-dm/Evaluation"
os.makedirs(output_dir_evaluation, exist_ok=True)
tuned_results_path = os.path.join(output_dir_evaluation, "smote_tuned_models_final_performance.xlsx")

X_sample = X.sample(frac=0.20, random_state=42)
y_sample = y.loc[X_sample.index]
print(f"Tuning will be performed on a sample of {X_sample.shape[0]} rows.")

top_4_models = ["Random Forest", "Decision Tree", "XGBoost", "Neural Network (MLP)"]
print(f"Top 4 models to be tuned: {top_4_models}")

param_grids = {
    "Random Forest": {'model__n_estimators': randint(100, 400), 'model__max_depth': randint(20, 50)},
    "Decision Tree": {'model__max_depth': randint(10, 40), 'model__criterion': ['gini', 'entropy']},
    "XGBoost": {'model__n_estimators': randint(200, 500), 'model__max_depth': randint(8, 16)},
    "Neural Network (MLP)": {'model__hidden_layer_sizes': [(50,), (100,)], 'model__alpha': [0.0001, 0.001]}
}
models_to_tune_all = {
    "Random Forest": RandomForestClassifier(random_state=42, n_jobs=-1), "Decision Tree": DecisionTreeClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, n_jobs=-1, eval_metric='mlogloss', use_label_encoder=False),
    "Neural Network (MLP)": MLPClassifier(random_state=42, max_iter=500, early_stopping=True)
}
models_to_tune = {name: models_to_tune_all[name] for name in top_4_models}

CV_FOLDS_TUNING = 3
kfold_tuning = StratifiedKFold(n_splits=CV_FOLDS_TUNING, shuffle=True, random_state=42)
N_ITER_SEARCH = 15
best_params_map = {}

# Define the resampling pipeline for tuning
resampling_pipeline_steps = [
    ('undersampling', RandomUnderSampler(sampling_strategy='not minority', random_state=42)),
    ('oversampling', SMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=1)),
]
resampling_pipeline = ImblearnPipeline(steps=resampling_pipeline_steps)

for name, model in models_to_tune.items():
    print(f"\n>>> Tuning model: {name}")
    start_time = time.time()
    # We create a final pipeline with scaling and the model
    full_pipeline = SklearnPipeline(steps=[('scaler', StandardScaler()), ('model', model)])

    # We must resample the data BEFORE passing it to RandomizedSearchCV
    # This is the only way to guarantee stability for XGBoost
    X_sample_res, y_sample_res = resampling_pipeline.fit_resample(X_sample, y_sample)

    random_search = RandomizedSearchCV(estimator=full_pipeline, param_distributions=param_grids.get(name, {}), n_iter=N_ITER_SEARCH, cv=kfold_tuning, scoring='f1_macro', n_jobs=-1, random_state=42, verbose=0)
    try:
        random_search.fit(X_sample_res, y_sample_res)
        best_params_map[name] = random_search.best_params_
        print(f"  -> Best F1-Score found on resampled sample: {random_search.best_score_:.4f}")
    except Exception as e:
        print(f"  -> Tuning failed for {name}. Error: {e}")
        best_params_map[name] = {}
    end_time = time.time()
    print(f"Finished tuning {name} in {(end_time - start_time) / 60:.2f} minutes.")

# ------------------------------------------------------------------------------
# PART 2: COMPREHENSIVE EVALUATION ON FULL DATASET
# ------------------------------------------------------------------------------
print("\n\n" + "="*60)
print("EVALUATION PHASE, PART 2: Final Comprehensive Evaluation on FULL Dataset")
print("="*60)

final_results = []
CV_FOLDS_FINAL = 5
kfold_final = StratifiedKFold(n_splits=CV_FOLDS_FINAL, shuffle=True, random_state=42)

for name, params in best_params_map.items():
    if not params:
        print(f"\n>>> Skipping final validation for {name} as tuning failed.")
        continue

    print(f"\n>>> Running final validation for tuned model: {name}")
    start_time = time.time()

    # Get the base model and set its tuned parameters
    final_model = models_to_tune_all[name]
    final_model.set_params(**{key.replace('model__', ''): val for key, val in params.items()})

    # The full pipeline for the final validation run
    final_pipeline = ImblearnPipeline(steps=[
        ('undersampling', RandomUnderSampler(sampling_strategy='not minority', random_state=42)),
        ('oversampling', SMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=3)),
        ('scaler', StandardScaler()),
        ('model', final_model)
    ])

    # Use cross_validate with the imblearn pipeline
    scores = cross_validate(final_pipeline, X, y, cv=kfold_final, scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'], n_jobs=-1)

    end_time = time.time()
    final_results.append({
        'Model': f"{name} (Tuned)",
        'Accuracy': np.mean(scores['test_accuracy']),
        'Precision': np.mean(scores['test_precision_macro']),
        'Recall': np.mean(scores['test_recall_macro']),
        'F1-Score': np.mean(scores['test_f1_macro'])
    })
    print(f"Finished final validation for {name} in {(end_time - start_time) / 60:.2f} minutes.")


# ------------------------------------------------------------------------------
# PART 3: FINAL REPORT
# ------------------------------------------------------------------------------
print("\n\n" + "="*70)
print("FINAL COMPREHENSIVE REPORT (TUNED MODELS WITH COMBINED SAMPLING)")
print("="*70)

if final_results:
    final_report_df = pd.DataFrame(final_results)
    final_report_df = final_report_df.sort_values(by='F1-Score', ascending=False)
    print(final_report_df.to_string(index=False, float_format="%.4f"))
else:
    print("No models were successfully tuned and evaluated.")

# ------------------------------------------------------------------------------
# PART 4: SAVE THE FINAL REPORT TO AN EXCEL FILE
# ------------------------------------------------------------------------------
if final_results:
    try:
        # We also save the dictionary of best parameters for future reference
        params_df = pd.DataFrame.from_dict(best_params_map, orient='index')

        with pd.ExcelWriter(tuned_results_path) as writer:
            final_report_df.to_excel(writer, sheet_name='Tuned_Model_Performance', index=False)
            params_df.to_excel(writer, sheet_name='Best_Parameters_Found')

        print(f"\n\nSUCCESS: Full tuning report and parameters saved to:")
        print(tuned_results_path)
    except Exception as e:
        print(f"\n\nERROR: Could not save the tuning results file. Error: {e}")

**APPROACH THREE : Data Augmentation with BORDERLINE-SMOTE**

In [ ]:
# ==============================================================================
# EVALUATION PHASE - BORDERLINE-SMOTE BASELINE EVALUATION FINE-TUNING
# ==============================================================================
# This cell is fully self-contained and robust. It takes the top models from the
# Borderline-SMOTE baseline and performs hyperparameter tuning.

# ------------------------------------------------------------------------------
# 0. IMPORTS FOR THIS CELL
# ------------------------------------------------------------------------------
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from scipy.stats import randint
from collections import Counter
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# --- Define Paths ---
# Input paths
data_prep_dir = "/workspace/Crisp-dm/Data_Preparation"
modeling_dir = "/workspace/Crisp-dm/Modeling"
cleaned_data_path = os.path.join(data_prep_dir, "final_cleaned_encoded_dataset.xlsx")
borderline_baseline_path = os.path.join(modeling_dir, "borderline_smote_baseline_results.xlsx")

# Output path for this cell's results
evaluation_dir = "/workspace/Crisp-dm/Evaluation"
os.makedirs(evaluation_dir, exist_ok=True)
tuned_results_path_borderline = os.path.join(evaluation_dir, "borderline_smote_tuned_models_final_performance.xlsx")

# ------------------------------------------------------------------------------
# 1. LOAD PREREQUISITE DATA
# ------------------------------------------------------------------------------
print("\n" + "="*60)
print("Loading prerequisite data for Borderline-SMOTE tuning...")
print("="*60)
try:
    df = pd.read_excel(cleaned_data_path)
    results_df_borderline = pd.read_excel(borderline_baseline_path)
    print("Successfully loaded cleaned dataset and baseline results.")
except FileNotFoundError as e:
    print(f"ERROR: Prerequisite file not found: {e.filename}")
    print("Please ensure Cell 1 and the Borderline-SMOTE modeling cell have been run successfully.")
    exit()

# Prepare data and get the top 4 models list
X = df.drop('Label', axis=1)
y = df['Label']
top_4_models_borderline = results_df_borderline.head(4)['Model'].tolist()

# ------------------------------------------------------------------------------
# 2. FAST HYPERPARAMETER TUNING ON A DATA SAMPLE
# ------------------------------------------------------------------------------
print("\n\n" + "="*60)
print("EVALUATION: Finding Best Hyperparameters for Top Borderline-SMOTE Models")
print("="*60)

X_sample = X.sample(frac=0.20, random_state=42)
y_sample = y.loc[X_sample.index]
print(f"Tuning will be performed on a sample of {X_sample.shape[0]} rows.")
print(f"Top models to be tuned (from baseline file): {top_4_models_borderline}")

# Define parameter grids
param_grids = {
    "Random Forest": {'n_estimators': randint(100, 400), 'max_depth': randint(20, 50)},
    "Decision Tree": {'max_depth': randint(10, 40), 'criterion': ['gini', 'entropy']},
    "XGBoost": {'n_estimators': randint(200, 500), 'max_depth': randint(8, 16)},
    "Neural Network (MLP)": {'hidden_layer_sizes': [(50,), (100,)], 'alpha': [0.0001, 0.001]}
}
# Base model instances for tuning
models_to_tune_all = {
    "Random Forest": RandomForestClassifier(random_state=42, n_jobs=-1),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, n_jobs=-1, eval_metric='mlogloss', use_label_encoder=False),
    "Neural Network (MLP)": MLPClassifier(random_state=42, max_iter=500, early_stopping=True)
}
models_to_tune = {name: models_to_tune_all[name] for name in top_4_models_borderline}

CV_FOLDS_TUNING = 3
kfold_tuning = StratifiedKFold(n_splits=CV_FOLDS_TUNING, shuffle=True, random_state=42)
N_ITER_SEARCH = 15
best_params_map_borderline = {}

# --- Instantiate the resamplers and scaler directly ---
under_sampler = RandomUnderSampler(sampling_strategy='not minority', random_state=42)
over_sampler = BorderlineSMOTE(sampling_strategy='not majority', random_state=42, k_neighbors=3)
scaler = StandardScaler()

for name, model in models_to_tune.items():
    print(f"\n>>> Tuning model: {name} with Borderline-SMOTE")
    start_time = time.time()

    # --- Apply resampling and scaling directly (The Brute-Force Fix) ---
    try:
        X_under, y_under = under_sampler.fit_resample(X_sample, y_sample)
        X_resampled, y_resampled = over_sampler.fit_resample(X_under, y_under)
        X_resampled_scaled = scaler.fit_transform(X_resampled)
    except Exception as e:
        print(f"  -> Resampling failed for {name} on the sample. Skipping. Error: {e}")
        best_params_map_borderline[name] = {}
        continue # Move to the next model

    # --- Tune the model on the fully prepared data ---
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grids.get(name, {}), n_iter=N_ITER_SEARCH, cv=kfold_tuning, scoring='f1_macro', n_jobs=-1, random_state=42)
    try:
        # Encode labels just for this fit to ensure XGBoost is stable
        le_tune = LabelEncoder()
        y_resampled_encoded = le_tune.fit_transform(y_resampled)
        random_search.fit(X_resampled_scaled, y_resampled_encoded)
        best_params_map_borderline[name] = random_search.best_params_
        print(f"  -> Best F1-Score found on resampled sample: {random_search.best_score_:.4f}")
    except Exception as e:
        print(f"  -> Tuning failed for {name}. Error: {e}")
        best_params_map_borderline[name] = {}
    end_time = time.time()
    print(f"Finished tuning {name} in {(end_time - start_time) / 60:.2f} minutes.")


# ------------------------------------------------------------------------------
# 3. DISPLAY TUNING RESULTS
# ------------------------------------------------------------------------------
print("\n\n" + "="*70)
print("BORDERLINE-SMOTE HYPERPARAMETER TUNING COMPLETE")
print("="*70)
for name, params in best_params_map_borderline.items():
    if params:
        print(f"Best parameters for {name}:")
        print(params)
    else:
        print(f"Tuning failed for {name}.")

# ------------------------------------------------------------------------------
# 4: FINAL REPORT AND SAVING
# ------------------------------------------------------------------------------
print("\n\n" + "="*80)
print("FINAL COMPREHENSIVE REPORT (TUNED BORDERLINE-SMOTE MODELS ON FULL DATA)")
print("="*80)

if final_results:
    final_report_df = pd.DataFrame(final_results)
    final_report_df = final_report_df.sort_values(by='F1-Score', ascending=False)
    print(final_report_df.to_string(index=False, float_format="%.4f"))

    # --- SAVE THE FINAL REPORT TO AN EXCEL FILE ---
    try:
        params_df = pd.DataFrame.from_dict(best_params_map_borderline, orient='index')
        with pd.ExcelWriter(tuned_results_path_borderline) as writer:
            final_report_df.to_excel(writer, sheet_name='Tuned_Model_Performance', index=False)
            params_df.to_excel(writer, sheet_name='Best_Parameters_Found')

        print(f"\n\nSUCCESS: Full tuning report and parameters saved to:")
        print(tuned_results_path_borderline)
    except Exception as e:
        print(f"\n\nERROR: Could not save the tuning results file. Error: {e}")
else:
    print("No models were successfully tuned and evaluated.")

**SMOTE VS BORDERLINE-SMOTE EVALUATION**

In [ ]:
# ==============================================================================
# CRISP-DM Phase: EVALUATION
# ==============================================================================
# This cell LOADS the saved results from the baseline experiments, compares them,
# declares a winning strategy, and SAVES a comprehensive comparison report.

import pandas as pd
import numpy as np
import os

# ------------------------------------------------------------------------------
# 1. DEFINE PATHS AND LOAD BASELINE RESULTS
# ------------------------------------------------------------------------------
print("="*70)
print("EVALUATION: COMPARING SMOTE vs. BORDERLINE-SMOTE PERFORMANCE")
print("="*70)

# Define input and output paths
modeling_dir = "/workspace/Crisp-dm/Modeling"
evaluation_dir = "/workspace/Crisp-dm/Evaluation"
os.makedirs(evaluation_dir, exist_ok=True)

# Correctly named input files
smote_results_path = os.path.join(modeling_dir, "smote_baseline_results.xlsx")
borderline_results_path = os.path.join(modeling_dir, "borderline_smote_baseline_results.xlsx")

# Output file for this cell's report
comparison_report_path = os.path.join(evaluation_dir, "smote_vs_borderline_comparison_report.xlsx")

# Load the results from the Excel files
try:
    results_df_smote = pd.read_excel(smote_results_path)
    results_df_borderline = pd.read_excel(borderline_results_path)
    print("Successfully loaded baseline results from Excel files.")
except FileNotFoundError as e:
    print(f"ERROR: Prerequisite result file not found: {e.filename}")
    print("Please ensure the baseline modeling cells (for both SMOTE and Borderline-SMOTE) have been run successfully to create these files.")
    exit()

# ------------------------------------------------------------------------------
# 2. COMPARE SMOTE vs. BORDERLINE-SMOTE BASELINE RESULTS
# ------------------------------------------------------------------------------
# Add a column to each dataframe to identify the method
results_df_smote['Method'] = 'SMOTE'
results_df_borderline['Method'] = 'Borderline-SMOTE'

# Combine the results into a single table for easy comparison
comparison_df = pd.concat([results_df_smote, results_df_borderline])

# Pivot the table to show F1-Scores side-by-side for a direct comparison
final_comparison_table = comparison_df.pivot_table(
    index='Model',
    columns='Method',
    values='F1-Score'
).sort_values(by=['Borderline-SMOTE', 'SMOTE'], ascending=False) # Sort by performance

print("\n--- F1-Score Comparison ---")
print(final_comparison_table.to_string(float_format="%.4f"))

# ------------------------------------------------------------------------------
# 3. DECLARE THE WINNING STRATEGY
# ------------------------------------------------------------------------------
# Compare the F1-Score of the best model from each method
best_smote_score = results_df_smote['F1-Score'].max()
best_smote_model = results_df_smote.iloc[0]['Model']
best_borderline_score = results_df_borderline['F1-Score'].max()
best_borderline_model = results_df_borderline.iloc[0]['Model']

print("\n--- Conclusion ---")
print(f"Best SMOTE Model:      {best_smote_model} with F1-Score: {best_smote_score:.4f}")
print(f"Best Borderline-SMOTE Model: {best_borderline_model} with F1-Score: {best_borderline_score:.4f}")

if best_borderline_score > best_smote_score:
    winning_method_name = "Borderline-SMOTE"
    winning_df = results_df_borderline
    print(f"\nWINNER: Borderline-SMOTE provides a better performance for the top model.")
else:
    winning_method_name = "SMOTE"
    winning_df = results_df_smote
    print(f"\nWINNER: SMOTE provides a better or equal performance for the top model.")

# Select and store the top 3 models from the winning method's results
top_3_ensemble_models = winning_df.head(3)['Model'].tolist()

print(f"\nThe '{winning_method_name}' resampling method has been selected.")
print(f"The top 3 models for the ensemble are: {top_3_ensemble_models}")
print("\nProceed to the next cell to build the final ensemble model.")

# ------------------------------------------------------------------------------
# 4. SAVE THE COMPREHENSIVE COMPARISON REPORT
# ------------------------------------------------------------------------------
try:
    with pd.ExcelWriter(comparison_report_path) as writer:
        # Save the pivoted side-by-side table for a quick view
        final_comparison_table.to_excel(writer, sheet_name='Side-by-Side_Comparison')

        # Save the full combined data with all metrics for detailed analysis
        full_comparison_table = comparison_df.set_index(['Method', 'Model'])
        full_comparison_table.to_excel(writer, sheet_name='Combined_Raw_Data')

    print(f"\n\nSUCCESS: Full comparison report saved to:")
    print(comparison_report_path)
except Exception as e:
    print(f"\n\nERROR: Could not save the comparison report file. Error: {e}")

# DEPLOYMENT

**FINAL COMPARISON, ENSEMBLE LEARNING, AND DEPLOYMENT**

In [ ]:
# ==============================================================================
# FINAL COMPARISON, ENSEMBLE BUILDING, AND DEPLOYMENT
# ==============================================================================
# This self-contained cell is the final step. It loads all results and models,
# compares methods, and builds the final deployable ensemble model.

# ------------------------------------------------------------------------------
# 0. REQUIRED LIBRARIES FOR THIS CELL
# ------------------------------------------------------------------------------
import time
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# ------------------------------------------------------------------------------
# 1. LOAD ALL PREREQUISITE DATA AND RESULTS
# ------------------------------------------------------------------------------
print("\n" + "="*70)
print("EVALUATION & DEPLOYMENT: Loading all necessary data and results...")
print("="*70)

# Define paths
modeling_dir = "/workspace/Crisp-dm/Modeling"
evaluation_dir = "/workspace/Crisp-dm/Evaluation"
deployment_dir = "/workspace/Crisp-dm/Deployment"
os.makedirs(deployment_dir, exist_ok=True)
data_prep_dir = "/workspace/Crisp-dm/Data_Preparation"
cleaned_data_path = os.path.join(data_prep_dir, "final_cleaned_encoded_dataset.xlsx")
smote_results_path = os.path.join(modeling_dir, "smote_baseline_results.xlsx")
borderline_results_path = os.path.join(modeling_dir, "borderline_smote_baseline_results.xlsx")

try:
    results_df_smote = pd.read_excel(smote_results_path)
    results_df_borderline = pd.read_excel(borderline_results_path)
    print("Successfully loaded baseline result files.")
    df_clean = pd.read_excel(cleaned_data_path)
    X = df_clean.drop('Label', axis=1)
    y = df_clean['Label']
    print(f"Successfully loaded cleaned dataset. Shape: {df_clean.shape}")
except FileNotFoundError as e:
    print(f"ERROR: A prerequisite file was not found: {e.filename}")
    print("Please ensure Cells 1, 2, and 3 have been run successfully.")
    exit()

# ------------------------------------------------------------------------------
# 2. EVALUATION: COMPARE METHODS AND DECLARE WINNER
# ------------------------------------------------------------------------------
results_df_smote['Method'] = 'SMOTE'
results_df_borderline['Method'] = 'Borderline-SMOTE'
comparison_df = pd.concat([results_df_smote, results_df_borderline])
final_comparison_table = comparison_df.pivot_table(index='Model', columns='Method', values='F1-Score').sort_values(by=['Borderline-SMOTE', 'SMOTE'], ascending=False)

print("\n--- F1-Score Comparison ---")
print(final_comparison_table.to_string(float_format="%.4f"))

best_smote_score = results_df_smote['F1-Score'].max()
best_borderline_score = results_df_borderline['F1-Score'].max()

if best_borderline_score > best_smote_score:
    winning_method_name = "Borderline-SMOTE"
    winning_df = results_df_borderline
    print(f"\nWINNER: Borderline-SMOTE (Top F1: {best_borderline_score:.4f})")
else:
    winning_method_name = "SMOTE"
    winning_df = results_df_smote
    print(f"\nWINNER: SMOTE (Top F1: {best_smote_score:.4f})")

top_3_ensemble_models = winning_df.head(3)['Model'].tolist()
print(f"Top 3 models selected for ensemble: {top_3_ensemble_models}")


# ------------------------------------------------------------------------------
# 3. DEPLOYMENT: BUILD ENSEMBLE FROM PRE-TRAINED MODELS
# ------------------------------------------------------------------------------
print("\n" + "="*70)
print(f"BUILDING FINAL ENSEMBLE FROM PRE-TRAINED '{winning_method_name}' MODELS")
print("="*70)

ensemble_estimators = []
for model_name in top_3_ensemble_models:
    # Create the unique name for the estimator tuple
    estimator_name = model_name.lower().replace(' ', '_').replace('-', '')

    # Construct the filename based on the winning method and model name
    filename = f"{winning_method_name.lower()}_{estimator_name}.pkl"
    model_path = os.path.join(modeling_dir, filename)

    try:
        print(f"Loading model: {filename}...")
        loaded_model_pipeline = joblib.load(model_path)
        # The estimator for the VotingClassifier is the full pipeline
        ensemble_estimators.append((estimator_name, loaded_model_pipeline))
    except FileNotFoundError:
        print(f"ERROR: Could not find the pre-trained model file: {model_path}")
        print("Please ensure the corresponding modeling cell was run successfully.")
        continue

if len(ensemble_estimators) != len(top_3_ensemble_models):
    print("\nERROR: Could not load all top 3 models. Halting ensemble construction.")
    exit()

# Create the Voting Classifier using the loaded, pre-trained pipelines
# We are now ensembling the full pipelines themselves.
ensemble_model = VotingClassifier(estimators=ensemble_estimators, voting='hard')

# ------------------------------------------------------------------------------
# 4. EVALUATE THE FINAL ENSEMBLE MODEL
# ------------------------------------------------------------------------------
# We do not need to re-train the ensemble on the full dataset because the
# components are already fully trained pipelines. We can evaluate it directly.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# The 'fit' step for a VotingClassifier of pre-trained models is very fast.
# It simply validates the estimators.
print(f"\nFitting the final ensemble...")
start_time = time.time()
ensemble_model.fit(X_train, y_train) # Fit on a subset of data to initialize
end_time = time.time()
print(f"Ensemble fitting complete in {end_time - start_time:.2f} seconds.")

print("\n" + "="*70)
print("FINAL ENSEMBLE MODEL PERFORMANCE ON HOLD-OUT TEST SET")
print("="*70)

y_pred_final = ensemble_model.predict(X_test)
print(classification_report(y_test, y_pred_final))

# ------------------------------------------------------------------------------
# 5. SAVE THE DEPLOYABLE MODEL
# ------------------------------------------------------------------------------
model_filename = f"final_best_of_best_ensemble.pkl"
model_path = os.path.join(deployment_dir, model_filename)
try:
    joblib.dump(ensemble_model, model_path, compress=3)
    print(f"\nSUCCESS: Final deployable ensemble model saved to: {model_path}")
except Exception as e:
    print(f"\nERROR: Could not save the final model. Error: {e}")